<a href="https://colab.research.google.com/github/currencyfxjle/Knapsack_GDF_16/blob/main/Knapsack_GDF_16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.1/165.1 kB 2.4 MB/s eta 0:00:00


In [11]:
import gspread
from google.colab import auth
from google.auth import default
import pandas as pd

# Authenticate and Access Google Sheet
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Open Google Sheet
title = "2025 Project Agenda"
sheet = gc.open(title)

# ------------------------------------------------
# 1. Fetch and Clean Supply Data
# ------------------------------------------------
def fetch_supply():
    ws = sheet.worksheet("Supply")
    data = ws.get_all_values()
    supply_df = pd.DataFrame(data[1:], columns=data[0])

    # Clean and prepare data
    supply_df = supply_df[supply_df['Take into consideration?'].str.lower() == 'y']
    supply_df['Est Cash Amount'] = pd.to_numeric(
        supply_df['Est Cash Amount'].replace({',': ''}, regex=True)
    )
    supply_df['Project_date'] = pd.to_datetime(
        supply_df['Project_date'], errors='coerce'
    )

    # Convert GDF Profit to numeric (e.g., '25%' to 0.25)
    supply_df['GDF Profit'] = (
        supply_df['GDF Profit'].str.rstrip('%').astype(float) / 100
    )
    supply_df = supply_df.dropna()

    # ------------------------------------------------
    # **NEW:** Ensure Priority is numeric
    # ------------------------------------------------
    supply_df['Priority'] = pd.to_numeric(supply_df['Priority'], errors='coerce')
    supply_df = supply_df.dropna(subset=['Priority'])  # Remove rows with invalid Priority

    # -------------------------------
    # **UPDATED SORTING:** Sort by Priority ASC, GDF Profit DESC,
    # then Est Cash Amount DESC, then Project_date ASC
    # -------------------------------
    supply_df = supply_df.sort_values(
        by=['Priority', 'GDF Profit', 'Est Cash Amount', 'Project_date'],
        ascending=[True, False, False, True]
    )

    # Add additional tracking columns
    supply_df['Original Est Cash Amount'] = supply_df['Est Cash Amount']
    supply_df['Fully Allocated'] = False

    # Add category based on Est Cash Amount
    supply_df['Project Category'] = pd.cut(
        supply_df['Est Cash Amount'],
        bins=[0, 350000, 500000, 800000, 1000000, float('inf')],
        labels=['0-350k', '350k-500k', '500k-800k', '800k-1M', '1M+']
    )

    # Include project state for info
    supply_df['Project State'] = supply_df['State']

    return supply_df

# ------------------------------------------------
# 2. Fetch and Clean Demand Data
# ------------------------------------------------
def fetch_demand():
    ws = sheet.worksheet("Demand")
    data = ws.get_all_values()
    demand_df = pd.DataFrame(data[1:], columns=data[0])

    # Rename 'purchaser_name' to 'Purchaser Name' for consistency
    demand_df = demand_df.rename(columns={'purchaser_name': 'Purchaser Name'})

    # Clean and prepare data
    demand_df['approximated_appetite'] = pd.to_numeric(
        demand_df['approximated_appetite'].replace({',': ''}, regex=True)
    )
    demand_df['pending_amount'] = pd.to_numeric(
        demand_df['pending_amount'].replace({',': ''}, regex=True)
    )
    demand_df['Purchaser_Date'] = pd.to_datetime(
        demand_df['Purchaser_Date'], errors='coerce'
    )
    demand_df = demand_df.dropna()

    # Track remaining appetite
    demand_df['remaining_appetite'] = demand_df['approximated_appetite']

    # Add category based on approximated appetite
    demand_df['Purchaser Category'] = pd.cut(
        demand_df['approximated_appetite'],
        bins=[0, 350000, 500000, 800000, 1000000, float('inf')],
        labels=['0-350k', '350k-500k', '500k-800k', '800k-1M', '1M+']
    )

    # Add priority for state (CA=1, others=2)
    demand_df['State Priority'] = demand_df['State'].apply(
        lambda x: 1 if x == 'CA' else 2
    )

    # (Optional) Sort demand initially by appetite desc, then state priority
    demand_df = demand_df.sort_values(
        by=['approximated_appetite', 'State Priority'],
        ascending=[False, True]
    )

    return demand_df

# ------------------------------------------------
# 3. Match Supply to Demand without Financing
# ------------------------------------------------
def match_supply_to_demand(supply_df, demand_df):
    """
    Allocates chunks from supply to purchasers in demand.
    Prioritizes Priority (ascending),
    then GDF Profit (descending),
    then Est Cash Amount (descending) on supply side,
    and CA-first + large appetites on the demand side.

    Only allocates if purchaser's remaining appetite >= chunk cost.
    """
    allocations = []
    total_allocated = 0.0

    # Group supply by Project Name in the sorted order
    grouped_supply = supply_df.groupby('Project Name', sort=False)

    for project_name, group in grouped_supply:
        # If the entire project is already allocated, skip
        if group['Fully Allocated'].all():
            continue

        project_total_cost = group['Est Cash Amount'].sum()
        project_chunks = group.to_dict('records')  # Convert group to list of chunk-rows

        can_allocate_project = True
        temp_allocations = []
        temp_demand = demand_df.copy()  # Work on a copy, finalize only if successful

        # -------------------------------------
        # For each chunk in the current project
        # -------------------------------------
        for chunk in project_chunks:
            if chunk['Fully Allocated']:
                continue  # Already allocated

            chunk_cost = chunk['Est Cash Amount']
            allocated = False

            # Sort potential purchasers by: CA first, then large appetite
            # (State Priority asc => CA=1 first, then appetite desc)
            purchasers = temp_demand[temp_demand['remaining_appetite'] >= chunk_cost].sort_values(
                by=['State Priority', 'remaining_appetite'],
                ascending=[True, False]
            ).to_dict('records')

            # -------------------------------------
            # Try each purchaser in sorted order
            # -------------------------------------
            for purchaser in purchasers:
                # Must meet date requirement
                if purchaser['Purchaser_Date'] > chunk['Project_date']:
                    # This purchaser isn't eligible yet
                    continue

                # Current stats
                p_appetite = purchaser['remaining_appetite']
                p_pending = purchaser['pending_amount']

                # Allocate the full chunk if possible
                new_appetite = p_appetite - chunk_cost
                new_pending = p_pending - chunk_cost

                # Clamp so they never go below zero
                if new_appetite < 0:
                    new_appetite = 0
                if new_pending < 0:
                    new_pending = 0

                # Update the temp purchaser
                purchaser['remaining_appetite'] = new_appetite
                purchaser['pending_amount'] = new_pending

                # Record the allocation
                temp_allocations.append({
                    "Purchaser Name": purchaser['Purchaser Name'],
                    "Purchaser State": purchaser['State'],
                    "Purchaser Date": purchaser['Purchaser_Date'],  # Use updated date
                    "Purchaser Category": purchaser['Purchaser Category'],
                    "Project ID": chunk.get('Project ID', ''),
                    "Project Name": project_name,
                    "Project Date": chunk['Project_date'],
                    "Project Category": chunk['Project Category'],
                    "Project State": chunk['Project State'],
                    "GDF Profit": chunk['GDF Profit'],
                    "Priority": chunk['Priority'],  # Include Priority in allocations
                    "Allocated Amount": chunk_cost,
                    "Financed Amount": 0.0,  # No financing
                    "Remaining Appetite After": new_appetite,
                    "Original Appetite": purchaser['approximated_appetite']
                })

                # Update the temp_demand DataFrame
                temp_demand.loc[
                    temp_demand['Purchaser Name'] == purchaser['Purchaser Name'],
                    'remaining_appetite'
                ] = new_appetite
                temp_demand.loc[
                    temp_demand['Purchaser Name'] == purchaser['Purchaser Name'],
                    'pending_amount'
                ] = new_pending

                # Mark chunk as allocated
                chunk['Fully Allocated'] = True
                allocated = True
                break  # Move to the next chunk after successful allocation

            # If we failed to allocate this chunk to any purchaser, skip the project
            if not allocated:
                can_allocate_project = False
                break

        # If every chunk is allocated, we finalize the temp updates
        if can_allocate_project:
            allocations.extend(temp_allocations)
            total_allocated += project_total_cost
            demand_df = temp_demand  # Make the changes permanent
            # Mark all chunks in this group as fully allocated
            supply_df.loc[group.index, 'Fully Allocated'] = True

    return total_allocated, allocations, supply_df, demand_df

# ------------------------------------------------
# 4. Main Execution
# ------------------------------------------------
def main():
    # 4.1 Read data
    supply_df = fetch_supply()
    demand_df = fetch_demand()

    # 4.2 Run allocation without financing
    total_allocated, allocations, remaining_supply, remaining_demand_df = match_supply_to_demand(
        supply_df,
        demand_df
    )

    # 4.3 Calculate summaries
    original_total_appetite = demand_df['approximated_appetite'].sum()
    remaining_demand = remaining_demand_df['remaining_appetite'].sum()

    original_total_project_value = supply_df['Original Est Cash Amount'].sum()
    remaining_supply_value = remaining_supply[remaining_supply['Fully Allocated'] == False]['Est Cash Amount'].sum()

    # Calculate percentage of allocated supply as a decimal
    percent_allocated_supply = (total_allocated / original_total_project_value) if original_total_project_value > 0 else 0

    # 4.4 Print Results Summary in Desired Format
    print("\nResults Summary:")
    print(f"Total Allocated \t {total_allocated:,.2f}")
    print(f"DEMAND\t SUM")
    print(f"Original Total Appetite\t {original_total_appetite:,.2f}")
    print(f"Remaining Demand\t {remaining_demand:,.2f}")
    print()
    print(f"% of Allocated Supply\t{percent_allocated_supply:.2f}")
    print(f"SUPPLY\t SUM")
    print(f"Original Total Project Value\t {original_total_project_value:,.2f}")
    print(f"Remaining Supply\t {remaining_supply_value:,.2f}")

    # 4.5 Create Supply Summary
    supply_summary = supply_df.groupby('Project Name').agg(
        Total_Chunks_Before=('Est Cash Amount', 'count'),
        Total_Chunks_After=('Fully Allocated', lambda x: x.sum()),
        Total_Project_Value=('Est Cash Amount', 'sum'),
        Latest_Project_Date=('Project_date', 'max'),           # New Column
        Max_Est_Cash_Amount=('Est Cash Amount', 'max'),       # New Column
        GDF_Profit=('GDF Profit', 'max'),                      # New Column
        Priority=('Priority', 'max')                            # Include Priority in summary
    ).reset_index()

    # 4.6 Create Demand Summary (Renamed to Remaining Demand)
    allocations_df = pd.DataFrame(allocations)

    if not allocations_df.empty:
        # Group allocations by purchaser
        allocations_grouped = allocations_df.groupby('Purchaser Name').agg(
            Original_Appetite=('Original Appetite', 'first'),
            Total_Allocated=('Allocated Amount', 'sum'),
            Remaining_Appetite=('Remaining Appetite After', 'last'),
            Purchaser_Date=('Purchaser Date', 'first'),  # Use the latest date
            Purchaser_Category=('Purchaser Category', 'first')  # Include Purchaser Category
        ).reset_index()
    else:
        # If no allocations were made, create an empty grouped DataFrame
        allocations_grouped = pd.DataFrame(columns=[
            'Purchaser Name', 'Original_Appetite', 'Total_Allocated',
            'Remaining_Appetite', 'Purchaser_Date', 'Purchaser_Category'
        ])

    # Merge with original demand to include purchasers with no allocations
    demand_summary = demand_df[['Purchaser Name', 'approximated_appetite', 'pending_amount', 'Purchaser_Date', 'Purchaser Category']].rename(
        columns={
            'approximated_appetite': 'Original_Appetite',
            'pending_amount': 'Updated_Pending_Amount_Original',
            'Purchaser_Date': 'Purchaser_Date_original'  # Temporary column for merging
        }
    )

    # Perform merge with allocations_grouped to include allocation details
    demand_summary = demand_summary.merge(
        allocations_grouped[['Purchaser Name', 'Total_Allocated',
                             'Remaining_Appetite', 'Purchaser_Date', 'Purchaser_Category']],
        on='Purchaser Name',
        how='left',
        suffixes=('_original', '_alloc')
    )

    # Fill NaN for purchasers with no allocations
    demand_summary['Total_Allocated'] = demand_summary['Total_Allocated'].fillna(0)
    demand_summary['Remaining_Appetite'] = demand_summary['Remaining_Appetite'].fillna(demand_summary['Original_Appetite'])

    # Fill 'Purchaser Category' with allocation's category or original
    demand_summary['Purchaser Category'] = demand_summary['Purchaser_Category'].fillna(demand_summary['Purchaser Category'])

    # Handle Purchaser_Date by prioritizing allocations and filling with original dates
    demand_summary['Purchaser_Date'] = demand_summary['Purchaser_Date'].fillna(demand_summary['Purchaser_Date_original'])

    # Drop redundant 'Purchaser_Date_original', 'Updated_Pending_Amount_Original', and 'Purchaser_Category'
    demand_summary = demand_summary.drop(columns=['Purchaser_Date_original', 'Updated_Pending_Amount_Original', 'Purchaser_Category'])

    # Calculate percentages
    demand_summary['Percentage_Allocated'] = (
        (demand_summary['Total_Allocated'] / demand_summary['Original_Appetite']) * 100
    ).round(2)
    demand_summary['Percentage_Remaining'] = (
        (demand_summary['Remaining_Appetite'] / demand_summary['Original_Appetite']) * 100
    ).round(2)

    # 4.7 Create Summary DataFrame in Desired Format
    summary_data = pd.DataFrame([
        {"Metric": "Total Allocated", "Value": total_allocated},
        {"Metric": "DEMAND", "Value": "SUM"},
        {"Metric": "Original Total Appetite", "Value": original_total_appetite},
        {"Metric": "Remaining Demand", "Value": remaining_demand},
        {"Metric": "", "Value": ""},
        {"Metric": "% of Allocated Supply", "Value": percent_allocated_supply},
        {"Metric": "SUPPLY", "Value": "SUM"},
        {"Metric": "Original Total Project Value", "Value": original_total_project_value},
        {"Metric": "Remaining Supply", "Value": remaining_supply_value}
    ])

    # 4.8 Save everything to Excel
    with pd.ExcelWriter("allocation_results.xlsx", engine="xlsxwriter") as writer:
        # Allocations Sheet
        allocations_df.to_excel(writer, sheet_name="Allocations", index=False)

        # Summary Sheet
        summary_data.to_excel(writer, sheet_name="Summary", index=False)

        # Remaining Supply Sheet
        remaining_supply_cleaned = remaining_supply.drop(columns=['Original Est Cash Amount'])
        remaining_supply_cleaned.to_excel(writer, sheet_name="Remaining Supply", index=False)

        # Remaining Demand Sheet (Renamed from Demand Summary)
        remaining_demand_cleaned = demand_summary[['Purchaser Name', 'Purchaser Category', 'Original_Appetite', 'Total_Allocated', 'Remaining_Appetite', 'Purchaser_Date', 'Percentage_Allocated', 'Percentage_Remaining']]
        remaining_demand_cleaned.to_excel(writer, sheet_name="Remaining Demand", index=False)

        # Supply Summary Sheet
        supply_summary.to_excel(writer, sheet_name="Supply Summary", index=False)

    print("Results saved to allocation_results.xlsx")

# Execute the main function
if __name__ == "__main__":
    main()


Results Summary:
Total Allocated 	 42,281,020.23
DEMAND	 SUM
Original Total Appetite	 103,350,380.00
Remaining Demand	 61,069,359.77

% of Allocated Supply	0.67
SUPPLY	 SUM
Original Total Project Value	 62,822,512.63
Remaining Supply	 20,541,492.40
Results saved to allocation_results.xlsx


**Adding Gross Profit**

In [12]:
import gspread
from google.colab import auth
from google.auth import default
import pandas as pd

# Authenticate and Access Google Sheet
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Open Google Sheet
title = "2025 Project Agenda"
sheet = gc.open(title)

# ------------------------------------------------
# 1. Fetch and Clean Supply Data
# ------------------------------------------------
def fetch_supply():
    ws = sheet.worksheet("Supply")
    data = ws.get_all_values()
    supply_df = pd.DataFrame(data[1:], columns=data[0])

    # Clean and prepare data
    supply_df = supply_df[supply_df['Take into consideration?'].str.lower() == 'y']
    supply_df['Est Cash Amount'] = pd.to_numeric(
        supply_df['Est Cash Amount'].replace({',': ''}, regex=True)
    )
    supply_df['Project_date'] = pd.to_datetime(
        supply_df['Project_date'], errors='coerce'
    )

    # Convert GDF Profit to numeric (e.g., '25%' to 0.25)
    supply_df['GDF Profit'] = (
        supply_df['GDF Profit'].str.rstrip('%').astype(float) / 100
    )
    supply_df = supply_df.dropna()

    # ------------------------------------------------
    # **NEW:** Ensure Priority is numeric
    # ------------------------------------------------
    supply_df['Priority'] = pd.to_numeric(supply_df['Priority'], errors='coerce')
    supply_df = supply_df.dropna(subset=['Priority'])  # Remove rows with invalid Priority

    # -------------------------------
    # **UPDATED SORTING:** Sort by Priority ASC, GDF Profit DESC,
    # then Est Cash Amount DESC, then Project_date ASC
    # -------------------------------
    supply_df = supply_df.sort_values(
        by=['Priority', 'GDF Profit', 'Est Cash Amount', 'Project_date'],
        ascending=[True, False, False, True]
    )

    # Add additional tracking columns
    supply_df['Original Est Cash Amount'] = supply_df['Est Cash Amount']
    supply_df['Fully Allocated'] = False

    # Add category based on Est Cash Amount
    supply_df['Project Category'] = pd.cut(
        supply_df['Est Cash Amount'],
        bins=[0, 350000, 500000, 800000, 1000000, float('inf')],
        labels=['0-350k', '350k-500k', '500k-800k', '800k-1M', '1M+']
    )

    # Include project state for info
    supply_df['Project State'] = supply_df['State']

    return supply_df

# ------------------------------------------------
# 2. Fetch and Clean Demand Data
# ------------------------------------------------
def fetch_demand():
    ws = sheet.worksheet("Demand")
    data = ws.get_all_values()
    demand_df = pd.DataFrame(data[1:], columns=data[0])

    # Rename 'purchaser_name' to 'Purchaser Name' for consistency
    demand_df = demand_df.rename(columns={'purchaser_name': 'Purchaser Name'})

    # Clean and prepare data
    demand_df['approximated_appetite'] = pd.to_numeric(
        demand_df['approximated_appetite'].replace({',': ''}, regex=True)
    )
    demand_df['pending_amount'] = pd.to_numeric(
        demand_df['pending_amount'].replace({',': ''}, regex=True)
    )
    demand_df['Purchaser_Date'] = pd.to_datetime(
        demand_df['Purchaser_Date'], errors='coerce'
    )
    demand_df = demand_df.dropna()

    # Track remaining appetite
    demand_df['remaining_appetite'] = demand_df['approximated_appetite']

    # Add category based on approximated appetite
    demand_df['Purchaser Category'] = pd.cut(
        demand_df['approximated_appetite'],
        bins=[0, 350000, 500000, 800000, 1000000, float('inf')],
        labels=['0-350k', '350k-500k', '500k-800k', '800k-1M', '1M+']
    )

    # Add priority for state (CA=1, others=2)
    demand_df['State Priority'] = demand_df['State'].apply(
        lambda x: 1 if x == 'CA' else 2
    )

    # (Optional) Sort demand initially by appetite desc, then state priority
    demand_df = demand_df.sort_values(
        by=['approximated_appetite', 'State Priority'],
        ascending=[False, True]
    )

    return demand_df

# ------------------------------------------------
# 3. Match Supply to Demand without Financing
# ------------------------------------------------
def match_supply_to_demand(supply_df, demand_df):
    """
    Allocates chunks from supply to purchasers in demand.
    Prioritizes Priority (ascending),
    then GDF Profit (descending),
    then Est Cash Amount (descending) on supply side,
    and CA-first + large appetites on the demand side.

    Only allocates if purchaser's remaining appetite >= chunk cost.
    """
    allocations = []
    total_allocated = 0.0

    # Group supply by Project Name in the sorted order
    grouped_supply = supply_df.groupby('Project Name', sort=False)

    for project_name, group in grouped_supply:
        # If the entire project is already allocated, skip
        if group['Fully Allocated'].all():
            continue

        project_total_cost = group['Est Cash Amount'].sum()
        project_chunks = group.to_dict('records')  # Convert group to list of chunk-rows

        can_allocate_project = True
        temp_allocations = []
        temp_demand = demand_df.copy()  # Work on a copy, finalize only if successful

        # -------------------------------------
        # For each chunk in the current project
        # -------------------------------------
        for chunk in project_chunks:
            if chunk['Fully Allocated']:
                continue  # Already allocated

            chunk_cost = chunk['Est Cash Amount']
            allocated = False

            # Sort potential purchasers by: CA first, then large appetite
            # (State Priority asc => CA=1 first, then appetite desc)
            purchasers = temp_demand[temp_demand['remaining_appetite'] >= chunk_cost].sort_values(
                by=['State Priority', 'remaining_appetite'],
                ascending=[True, False]
            ).to_dict('records')

            # -------------------------------------
            # Try each purchaser in sorted order
            # -------------------------------------
            for purchaser in purchasers:
                # Must meet date requirement
                if purchaser['Purchaser_Date'] > chunk['Project_date']:
                    # This purchaser isn't eligible yet
                    continue

                # Current stats
                p_appetite = purchaser['remaining_appetite']
                p_pending = purchaser['pending_amount']

                # Allocate the full chunk if possible
                new_appetite = p_appetite - chunk_cost
                new_pending = p_pending - chunk_cost

                # Clamp so they never go below zero
                if new_appetite < 0:
                    new_appetite = 0
                if new_pending < 0:
                    new_pending = 0

                # Update the temp purchaser
                purchaser['remaining_appetite'] = new_appetite
                purchaser['pending_amount'] = new_pending

                # Record the allocation
                temp_allocations.append({
                    "Purchaser Name": purchaser['Purchaser Name'],
                    "Purchaser State": purchaser['State'],
                    "Purchaser Date": purchaser['Purchaser_Date'],  # Use updated date
                    "Purchaser Category": purchaser['Purchaser Category'],
                    "Project ID": chunk.get('Project ID', ''),
                    "Project Name": project_name,
                    "Project Date": chunk['Project_date'],
                    "Project Category": chunk['Project Category'],
                    "Project State": chunk['Project State'],
                    "GDF Profit": chunk['GDF Profit'],
                    "Priority": chunk['Priority'],  # Include Priority in allocations
                    "Allocated Amount": chunk_cost,
                    "Financed Amount": 0.0,  # No financing
                    "Remaining Appetite After": new_appetite,
                    "Original Appetite": purchaser['approximated_appetite']
                })

                # Update the temp_demand DataFrame
                temp_demand.loc[
                    temp_demand['Purchaser Name'] == purchaser['Purchaser Name'],
                    'remaining_appetite'
                ] = new_appetite
                temp_demand.loc[
                    temp_demand['Purchaser Name'] == purchaser['Purchaser Name'],
                    'pending_amount'
                ] = new_pending

                # Mark chunk as allocated
                chunk['Fully Allocated'] = True
                allocated = True
                break  # Move to the next chunk after successful allocation

            # If we failed to allocate this chunk to any purchaser, skip the project
            if not allocated:
                can_allocate_project = False
                break

        # If every chunk is allocated, we finalize the temp updates
        if can_allocate_project:
            allocations.extend(temp_allocations)
            total_allocated += project_total_cost
            demand_df = temp_demand  # Make the changes permanent
            # Mark all chunks in this group as fully allocated
            supply_df.loc[group.index, 'Fully Allocated'] = True

    return total_allocated, allocations, supply_df, demand_df

# ------------------------------------------------
# 4. Main Execution
# ------------------------------------------------
def main():
    # 4.1 Read data
    supply_df = fetch_supply()
    demand_df = fetch_demand()

    # 4.2 Run allocation without financing
    total_allocated, allocations, remaining_supply, remaining_demand_df = match_supply_to_demand(
        supply_df,
        demand_df
    )

    # 4.3 Calculate summaries
    original_total_appetite = demand_df['approximated_appetite'].sum()
    remaining_demand = remaining_demand_df['remaining_appetite'].sum()

    original_total_project_value = supply_df['Original Est Cash Amount'].sum()
    remaining_supply_value = remaining_supply[remaining_supply['Fully Allocated'] == False]['Est Cash Amount'].sum()

    # Calculate percentage of allocated supply as a decimal
    percent_allocated_supply = (total_allocated / original_total_project_value) if original_total_project_value > 0 else 0

    # 4.4 Print Results Summary in Desired Format
    print("\nResults Summary:")
    print(f"Total Allocated \t {total_allocated:,.2f}")
    print(f"DEMAND\t SUM")
    print(f"Original Total Appetite\t {original_total_appetite:,.2f}")
    print(f"Remaining Demand\t {remaining_demand:,.2f}")
    print()
    print(f"% of Allocated Supply\t{percent_allocated_supply:.2f}")
    print(f"SUPPLY\t SUM")
    print(f"Original Total Project Value\t {original_total_project_value:,.2f}")
    print(f"Remaining Supply\t {remaining_supply_value:,.2f}")

    # 4.5 Create Supply Summary
    supply_summary = supply_df.groupby('Project Name').agg(
        Total_Chunks_Before=('Est Cash Amount', 'count'),
        Total_Chunks_After=('Fully Allocated', lambda x: x.sum()),
        Total_Project_Value=('Est Cash Amount', 'sum'),
        Latest_Project_Date=('Project_date', 'max'),           # New Column
        Max_Est_Cash_Amount=('Est Cash Amount', 'max'),       # New Column
        GDF_Profit=('GDF Profit', 'max'),                      # New Column
        Priority=('Priority', 'max')                            # Include Priority in summary
    ).reset_index()

    # ------------------------------------------------
    # **NEW:** Calculate Gross Profit for Supply Summary
    # ------------------------------------------------
    supply_summary['Gross_Profit'] = supply_summary['Total_Project_Value'] * supply_summary['GDF_Profit']

    # 4.6 Create Demand Summary (Renamed to Remaining Demand)
    allocations_df = pd.DataFrame(allocations)

    if not allocations_df.empty:
        # Group allocations by purchaser
        allocations_grouped = allocations_df.groupby('Purchaser Name').agg(
            Original_Appetite=('Original Appetite', 'first'),
            Total_Allocated=('Allocated Amount', 'sum'),
            Remaining_Appetite=('Remaining Appetite After', 'last'),
            Purchaser_Date=('Purchaser Date', 'first'),  # Use the latest date
            Purchaser_Category=('Purchaser Category', 'first')  # Include Purchaser Category
        ).reset_index()
    else:
        # If no allocations were made, create an empty grouped DataFrame
        allocations_grouped = pd.DataFrame(columns=[
            'Purchaser Name', 'Original_Appetite', 'Total_Allocated',
            'Remaining_Appetite', 'Purchaser_Date', 'Purchaser_Category'
        ])

    # Merge with original demand to include purchasers with no allocations
    demand_summary = demand_df[['Purchaser Name', 'approximated_appetite', 'pending_amount', 'Purchaser_Date', 'Purchaser Category']].rename(
        columns={
            'approximated_appetite': 'Original_Appetite',
            'pending_amount': 'Updated_Pending_Amount_Original',
            'Purchaser_Date': 'Purchaser_Date_original'  # Temporary column for merging
        }
    )

    # Perform merge with allocations_grouped to include allocation details
    demand_summary = demand_summary.merge(
        allocations_grouped[['Purchaser Name', 'Total_Allocated',
                             'Remaining_Appetite', 'Purchaser_Date', 'Purchaser_Category']],
        on='Purchaser Name',
        how='left',
        suffixes=('_original', '_alloc')
    )

    # Fill NaN for purchasers with no allocations
    demand_summary['Total_Allocated'] = demand_summary['Total_Allocated'].fillna(0)
    demand_summary['Remaining_Appetite'] = demand_summary['Remaining_Appetite'].fillna(demand_summary['Original_Appetite'])

    # Fill 'Purchaser Category' with allocation's category or original
    demand_summary['Purchaser Category'] = demand_summary['Purchaser_Category'].fillna(demand_summary['Purchaser Category'])

    # Handle Purchaser_Date by prioritizing allocations and filling with original dates
    demand_summary['Purchaser_Date'] = demand_summary['Purchaser_Date'].fillna(demand_summary['Purchaser_Date_original'])

    # Drop redundant 'Purchaser_Date_original', 'Updated_Pending_Amount_Original', and 'Purchaser_Category'
    demand_summary = demand_summary.drop(columns=['Purchaser_Date_original', 'Updated_Pending_Amount_Original', 'Purchaser_Category'])

    # Calculate percentages
    demand_summary['Percentage_Allocated'] = (
        (demand_summary['Total_Allocated'] / demand_summary['Original_Appetite']) * 100
    ).round(2)
    demand_summary['Percentage_Remaining'] = (
        (demand_summary['Remaining_Appetite'] / demand_summary['Original_Appetite']) * 100
    ).round(2)

    # 4.7 Create Summary DataFrame in Desired Format
    summary_data = pd.DataFrame([
        {"Metric": "Total Allocated", "Value": total_allocated},
        {"Metric": "DEMAND", "Value": "SUM"},
        {"Metric": "Original Total Appetite", "Value": original_total_appetite},
        {"Metric": "Remaining Demand", "Value": remaining_demand},
        {"Metric": "", "Value": ""},
        {"Metric": "% of Allocated Supply", "Value": percent_allocated_supply},
        {"Metric": "SUPPLY", "Value": "SUM"},
        {"Metric": "Original Total Project Value", "Value": original_total_project_value},
        {"Metric": "Remaining Supply", "Value": remaining_supply_value}
    ])

    # 4.8 Save everything to Excel
    with pd.ExcelWriter("allocation_results.xlsx", engine="xlsxwriter") as writer:
        # Allocations Sheet
        allocations_df.to_excel(writer, sheet_name="Allocations", index=False)

        # Summary Sheet
        summary_data.to_excel(writer, sheet_name="Summary", index=False)

        # Remaining Supply Sheet
        remaining_supply_cleaned = remaining_supply.drop(columns=['Original Est Cash Amount'])
        remaining_supply_cleaned.to_excel(writer, sheet_name="Remaining Supply", index=False)

        # Remaining Demand Sheet (Renamed from Demand Summary)
        remaining_demand_cleaned = demand_summary[['Purchaser Name', 'Purchaser Category', 'Original_Appetite', 'Total_Allocated', 'Remaining_Appetite', 'Purchaser_Date', 'Percentage_Allocated', 'Percentage_Remaining']]
        remaining_demand_cleaned.to_excel(writer, sheet_name="Remaining Demand", index=False)

        # Supply Summary Sheet
        supply_summary.to_excel(writer, sheet_name="Supply Summary", index=False)

    print("Results saved to allocation_results.xlsx")

# Execute the main function
if __name__ == "__main__":
    main()



Results Summary:
Total Allocated 	 42,281,020.23
DEMAND	 SUM
Original Total Appetite	 103,350,380.00
Remaining Demand	 61,069,359.77

% of Allocated Supply	0.67
SUPPLY	 SUM
Original Total Project Value	 62,822,512.63
Remaining Supply	 20,541,492.40
Results saved to allocation_results.xlsx


**Adding Gross Profit metrics on Totals**

In [16]:
import gspread
from google.colab import auth
from google.auth import default
import pandas as pd

# Authenticate and Access Google Sheet
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Open Google Sheet
title = "2025 Project Agenda"
sheet = gc.open(title)

# ------------------------------------------------
# 1. Fetch and Clean Supply Data
# ------------------------------------------------
def fetch_supply():
    ws = sheet.worksheet("Supply")
    data = ws.get_all_values()
    supply_df = pd.DataFrame(data[1:], columns=data[0])

    # Clean and prepare data
    supply_df = supply_df[supply_df['Take into consideration?'].str.lower() == 'y']
    supply_df['Est Cash Amount'] = pd.to_numeric(
        supply_df['Est Cash Amount'].replace({',': ''}, regex=True)
    )
    supply_df['Project_date'] = pd.to_datetime(
        supply_df['Project_date'], errors='coerce'
    )

    # Convert GDF Profit to numeric (e.g., '25%' to 0.25)
    supply_df['GDF Profit'] = (
        supply_df['GDF Profit'].str.rstrip('%').astype(float) / 100
    )
    supply_df = supply_df.dropna()

    # ------------------------------------------------
    # **NEW:** Ensure Priority is numeric
    # ------------------------------------------------
    supply_df['Priority'] = pd.to_numeric(supply_df['Priority'], errors='coerce')
    supply_df = supply_df.dropna(subset=['Priority'])  # Remove rows with invalid Priority

    # -------------------------------
    # **UPDATED SORTING:** Sort by Priority ASC, GDF Profit DESC,
    # then Est Cash Amount DESC, then Project_date ASC
    # -------------------------------
    supply_df = supply_df.sort_values(
        by=['Priority', 'GDF Profit', 'Est Cash Amount', 'Project_date'],
        ascending=[True, False, False, True]
    )

    # Add additional tracking columns
    supply_df['Original Est Cash Amount'] = supply_df['Est Cash Amount']
    supply_df['Fully Allocated'] = False

    # Add category based on Est Cash Amount
    supply_df['Project Category'] = pd.cut(
        supply_df['Est Cash Amount'],
        bins=[0, 350000, 500000, 800000, 1000000, float('inf')],
        labels=['0-350k', '350k-500k', '500k-800k', '800k-1M', '1M+']
    )

    # Include project state for info
    supply_df['Project State'] = supply_df['State']

    return supply_df

# ------------------------------------------------
# 2. Fetch and Clean Demand Data
# ------------------------------------------------
def fetch_demand():
    ws = sheet.worksheet("Demand")
    data = ws.get_all_values()
    demand_df = pd.DataFrame(data[1:], columns=data[0])

    # Rename 'purchaser_name' to 'Purchaser Name' for consistency
    demand_df = demand_df.rename(columns={'purchaser_name': 'Purchaser Name'})

    # Clean and prepare data
    demand_df['approximated_appetite'] = pd.to_numeric(
        demand_df['approximated_appetite'].replace({',': ''}, regex=True)
    )
    demand_df['pending_amount'] = pd.to_numeric(
        demand_df['pending_amount'].replace({',': ''}, regex=True)
    )
    demand_df['Purchaser_Date'] = pd.to_datetime(
        demand_df['Purchaser_Date'], errors='coerce'
    )
    demand_df = demand_df.dropna()

    # Track remaining appetite
    demand_df['remaining_appetite'] = demand_df['approximated_appetite']

    # Add category based on approximated appetite
    demand_df['Purchaser Category'] = pd.cut(
        demand_df['approximated_appetite'],
        bins=[0, 350000, 500000, 800000, 1000000, float('inf')],
        labels=['0-350k', '350k-500k', '500k-800k', '800k-1M', '1M+']
    )

    # Add priority for state (CA=1, others=2)
    demand_df['State Priority'] = demand_df['State'].apply(
        lambda x: 1 if x == 'CA' else 2
    )

    # (Optional) Sort demand initially by appetite desc, then state priority
    demand_df = demand_df.sort_values(
        by=['approximated_appetite', 'State Priority'],
        ascending=[False, True]
    )

    return demand_df

# ------------------------------------------------
# 3. Match Supply to Demand without Financing
# ------------------------------------------------
def match_supply_to_demand(supply_df, demand_df):
    """
    Allocates chunks from supply to purchasers in demand.
    Prioritizes Priority (ascending),
    then GDF Profit (descending),
    then Est Cash Amount (descending) on supply side,
    and CA-first + large appetites on the demand side.

    Only allocates if purchaser's remaining appetite >= chunk cost.
    """
    allocations = []
    total_allocated = 0.0

    # Group supply by Project Name in the sorted order
    grouped_supply = supply_df.groupby('Project Name', sort=False)

    for project_name, group in grouped_supply:
        # If the entire project is already allocated, skip
        if group['Fully Allocated'].all():
            continue

        project_total_cost = group['Est Cash Amount'].sum()
        project_chunks = group.to_dict('records')  # Convert group to list of chunk-rows

        can_allocate_project = True
        temp_allocations = []
        temp_demand = demand_df.copy()  # Work on a copy, finalize only if successful

        # -------------------------------------
        # For each chunk in the current project
        # -------------------------------------
        for chunk in project_chunks:
            if chunk['Fully Allocated']:
                continue  # Already allocated

            chunk_cost = chunk['Est Cash Amount']
            allocated = False

            # Sort potential purchasers by: CA first, then large appetite
            # (State Priority asc => CA=1 first, then appetite desc)
            purchasers = temp_demand[temp_demand['remaining_appetite'] >= chunk_cost].sort_values(
                by=['State Priority', 'remaining_appetite'],
                ascending=[True, False]
            ).to_dict('records')

            # -------------------------------------
            # Try each purchaser in sorted order
            # -------------------------------------
            for purchaser in purchasers:
                # Must meet date requirement
                if purchaser['Purchaser_Date'] > chunk['Project_date']:
                    # This purchaser isn't eligible yet
                    continue

                # Current stats
                p_appetite = purchaser['remaining_appetite']
                p_pending = purchaser['pending_amount']

                # Allocate the full chunk if possible
                new_appetite = p_appetite - chunk_cost
                new_pending = p_pending - chunk_cost

                # Clamp so they never go below zero
                if new_appetite < 0:
                    new_appetite = 0
                if new_pending < 0:
                    new_pending = 0

                # Update the temp purchaser
                purchaser['remaining_appetite'] = new_appetite
                purchaser['pending_amount'] = new_pending

                # Record the allocation
                temp_allocations.append({
                    "Purchaser Name": purchaser['Purchaser Name'],
                    "Purchaser State": purchaser['State'],
                    "Purchaser Date": purchaser['Purchaser_Date'],  # Use updated date
                    "Purchaser Category": purchaser['Purchaser Category'],
                    "Project ID": chunk.get('Project ID', ''),
                    "Project Name": project_name,
                    "Project Date": chunk['Project_date'],
                    "Project Category": chunk['Project Category'],
                    "Project State": chunk['Project State'],
                    "GDF Profit": chunk['GDF Profit'],
                    "Priority": chunk['Priority'],  # Include Priority in allocations
                    "Allocated Amount": chunk_cost,
                    "Financed Amount": 0.0,  # No financing
                    "Remaining Appetite After": new_appetite,
                    "Original Appetite": purchaser['approximated_appetite']
                })

                # Update the temp_demand DataFrame
                temp_demand.loc[
                    temp_demand['Purchaser Name'] == purchaser['Purchaser Name'],
                    'remaining_appetite'
                ] = new_appetite
                temp_demand.loc[
                    temp_demand['Purchaser Name'] == purchaser['Purchaser Name'],
                    'pending_amount'
                ] = new_pending

                # Mark chunk as allocated
                chunk['Fully Allocated'] = True
                allocated = True
                break  # Move to the next chunk after successful allocation

            # If we failed to allocate this chunk to any purchaser, skip the project
            if not allocated:
                can_allocate_project = False
                break

        # If every chunk is allocated, we finalize the temp updates
        if can_allocate_project:
            allocations.extend(temp_allocations)
            total_allocated += project_total_cost
            demand_df = temp_demand  # Make the changes permanent
            # Mark all chunks in this group as fully allocated
            supply_df.loc[group.index, 'Fully Allocated'] = True

    return total_allocated, allocations, supply_df, demand_df

# ------------------------------------------------
# 4. Main Execution
# ------------------------------------------------
def main():
    # 4.1 Read data
    supply_df = fetch_supply()
    demand_df = fetch_demand()

    # 4.2 Run allocation without financing
    total_allocated, allocations, remaining_supply, remaining_demand_df = match_supply_to_demand(
        supply_df,
        demand_df
    )

    # 4.3 Calculate summaries
    original_total_appetite = demand_df['approximated_appetite'].sum()
    remaining_demand = remaining_demand_df['remaining_appetite'].sum()

    original_total_project_value = supply_df['Original Est Cash Amount'].sum()
    remaining_supply_value = remaining_supply[remaining_supply['Fully Allocated'] == False]['Est Cash Amount'].sum()

    # Calculate percentage of allocated supply as a decimal
    percent_allocated_supply = (total_allocated / original_total_project_value) if original_total_project_value > 0 else 0

    # 4.4 Print Results Summary in Desired Format
    print("\nResults Summary:")
    print(f"Total Allocated \t {total_allocated:,.2f}")
    print(f"DEMAND\t SUM")
    print(f"Original Total Appetite\t {original_total_appetite:,.2f}")
    print(f"Remaining Demand\t {remaining_demand:,.2f}")
    print()
    print(f"% of Allocated Supply\t{percent_allocated_supply:.2f}")
    print(f"SUPPLY\t SUM")
    print(f"Original Total Project Value\t {original_total_project_value:,.2f}")
    print(f"Remaining Supply\t {remaining_supply_value:,.2f}")

    # 4.5 Create Supply Summary
    supply_summary = supply_df.groupby('Project Name').agg(
        Total_Chunks_Before=('Est Cash Amount', 'count'),
        Total_Chunks_After=('Fully Allocated', lambda x: x.sum()),
        Total_Project_Value=('Est Cash Amount', 'sum'),
        Latest_Project_Date=('Project_date', 'max'),           # New Column
        Max_Est_Cash_Amount=('Est Cash Amount', 'max'),       # New Column
        GDF_Profit=('GDF Profit', 'max'),                      # New Column
        Priority=('Priority', 'max')                            # Include Priority in summary
    ).reset_index()

    # ------------------------------------------------
    # **NEW:** Calculate Gross Profit for Supply Summary
    # ------------------------------------------------
    supply_summary['Gross_Profit'] = supply_summary['Total_Project_Value'] * supply_summary['GDF_Profit']

    # ------------------------------------------------
    # **NEW:** Calculate Gross Profit Metrics for Summary
    # ------------------------------------------------
    total_gross_profit = supply_summary['Gross_Profit'].sum()
    total_gross_profit_allocated = supply_summary[supply_summary['Total_Chunks_After'] > 0]['Gross_Profit'].sum()
    total_gross_profit_unallocated = supply_summary[supply_summary['Total_Chunks_After'] == 0]['Gross_Profit'].sum()

    # 4.6 Create Demand Summary (Renamed to Remaining Demand)
    allocations_df = pd.DataFrame(allocations)

    if not allocations_df.empty:
        # Group allocations by purchaser
        allocations_grouped = allocations_df.groupby('Purchaser Name').agg(
            Original_Appetite=('Original Appetite', 'first'),
            Total_Allocated=('Allocated Amount', 'sum'),
            Remaining_Appetite=('Remaining Appetite After', 'last'),
            Purchaser_Date=('Purchaser Date', 'first'),  # Use the latest date
            Purchaser_Category=('Purchaser Category', 'first')  # Include Purchaser Category
        ).reset_index()
    else:
        # If no allocations were made, create an empty grouped DataFrame
        allocations_grouped = pd.DataFrame(columns=[
            'Purchaser Name', 'Original_Appetite', 'Total_Allocated',
            'Remaining_Appetite', 'Purchaser_Date', 'Purchaser_Category'
        ])

    # Merge with original demand to include purchasers with no allocations
    demand_summary = demand_df[['Purchaser Name', 'approximated_appetite', 'pending_amount', 'Purchaser_Date', 'Purchaser Category']].rename(
        columns={
            'approximated_appetite': 'Original_Appetite',
            'pending_amount': 'Updated_Pending_Amount_Original',
            'Purchaser_Date': 'Purchaser_Date_original'  # Temporary column for merging
        }
    )

    # Perform merge with allocations_grouped to include allocation details
    demand_summary = demand_summary.merge(
        allocations_grouped[['Purchaser Name', 'Total_Allocated',
                             'Remaining_Appetite', 'Purchaser_Date', 'Purchaser_Category']],
        on='Purchaser Name',
        how='left',
        suffixes=('_original', '_alloc')
    )

    # Fill NaN for purchasers with no allocations
    demand_summary['Total_Allocated'] = demand_summary['Total_Allocated'].fillna(0)
    demand_summary['Remaining_Appetite'] = demand_summary['Remaining_Appetite'].fillna(demand_summary['Original_Appetite'])

    # Fill 'Purchaser Category' with allocation's category or original
    demand_summary['Purchaser Category'] = demand_summary['Purchaser_Category'].fillna(demand_summary['Purchaser Category'])

    # Handle Purchaser_Date by prioritizing allocations and filling with original dates
    demand_summary['Purchaser_Date'] = demand_summary['Purchaser_Date'].fillna(demand_summary['Purchaser_Date_original'])

    # Drop redundant 'Purchaser_Date_original', 'Updated_Pending_Amount_Original', and 'Purchaser_Category'
    demand_summary = demand_summary.drop(columns=['Purchaser_Date_original', 'Updated_Pending_Amount_Original', 'Purchaser_Category'])

    # Calculate percentages
    demand_summary['Percentage_Allocated'] = (
        (demand_summary['Total_Allocated'] / demand_summary['Original_Appetite']) * 100
    ).round(2)
    demand_summary['Percentage_Remaining'] = (
        (demand_summary['Remaining_Appetite'] / demand_summary['Original_Appetite']) * 100
    ).round(2)

    # 4.7 Create Summary DataFrame in Desired Format
    summary_data = pd.DataFrame([
        {"Metric": "Total Allocated", "Value": total_allocated},
        {"Metric": "DEMAND", "Value": "SUM"},
        {"Metric": "Original Total Appetite", "Value": original_total_appetite},
        {"Metric": "Remaining Demand", "Value": remaining_demand},
        {"Metric": "", "Value": ""},
        {"Metric": "% of Allocated Supply", "Value": percent_allocated_supply},
        {"Metric": "SUPPLY", "Value": "SUM"},
        {"Metric": "Original Total Project Value", "Value": original_total_project_value},
        {"Metric": "Remaining Supply", "Value": remaining_supply_value},
        {"Metric": "", "Value": ""},
        # **NEW:** Gross Profit Metrics
        {"Metric": "Total Gross Profit (all projects)", "Value": total_gross_profit},
        {"Metric": "Total Gross Profit Allocated", "Value": total_gross_profit_allocated},
        {"Metric": "Total Gross Profit Unallocated", "Value": total_gross_profit_unallocated}
    ])

    # 4.8 Save everything to Excel
    with pd.ExcelWriter("allocation_results.xlsx", engine="xlsxwriter") as writer:
        # Allocations Sheet
        allocations_df.to_excel(writer, sheet_name="Allocations", index=False)

        # Summary Sheet
        summary_data.to_excel(writer, sheet_name="Summary", index=False)

        # Remaining Supply Sheet
        remaining_supply_cleaned = remaining_supply.drop(columns=['Original Est Cash Amount'])
        remaining_supply_cleaned.to_excel(writer, sheet_name="Remaining Supply", index=False)

        # Remaining Demand Sheet (Renamed from Demand Summary)
        remaining_demand_cleaned = demand_summary[['Purchaser Name', 'Purchaser Category', 'Original_Appetite', 'Total_Allocated', 'Remaining_Appetite', 'Purchaser_Date', 'Percentage_Allocated', 'Percentage_Remaining']]
        remaining_demand_cleaned.to_excel(writer, sheet_name="Remaining Demand", index=False)

        # Supply Summary Sheet
        supply_summary.to_excel(writer, sheet_name="Supply Summary", index=False)

    print("Results saved to allocation_results.xlsx")

# Execute the main function
if __name__ == "__main__":
    main()



Results Summary:
Total Allocated 	 39,941,802.47
DEMAND	 SUM
Original Total Appetite	 120,414,628.00
Remaining Demand	 80,472,825.53

% of Allocated Supply	0.49
SUPPLY	 SUM
Original Total Project Value	 82,199,692.42
Remaining Supply	 42,257,889.95
Results saved to allocation_results.xlsx


**Gross Profit metrics to Bank Credit**

**corrections to metric calculations**

In [7]:
import gspread
from google.colab import auth
from google.auth import default
import pandas as pd
from collections import defaultdict

# -------------------------------------------------------------------------
# 0. Authenticate and open the Google Sheet
# -------------------------------------------------------------------------
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

title = "2025 Project Agenda"
sheet = gc.open(title)

# -------------------------------------------------------------------------
# 1. Fetch Supply
# -------------------------------------------------------------------------
def fetch_supply():
    ws = sheet.worksheet("Supply")
    data = ws.get_all_values()
    supply_df = pd.DataFrame(data[1:], columns=data[0])

    # Keep only rows marked 'Y'
    supply_df = supply_df[supply_df['Take into consideration?'].str.lower() == 'y']

    # Convert numeric fields
    supply_df['Est Cash Amount'] = pd.to_numeric(
        supply_df['Est Cash Amount'].replace({',': ''}, regex=True)
    )
    supply_df['Project_date'] = pd.to_datetime(supply_df['Project_date'], errors='coerce')

    # Convert GDF Profit to numeric (e.g., "25%" -> 0.25)
    supply_df['GDF Profit'] = (
        supply_df['GDF Profit'].str.rstrip('%').astype(float) / 100
    )

    # Drop rows missing critical columns
    supply_df = supply_df.dropna(subset=['Est Cash Amount','Project_date','GDF Profit'])

    # Ensure we have a Project ID
    if 'Project ID' not in supply_df.columns or supply_df['Project ID'].isnull().all():
        supply_df['Project ID'] = supply_df['Project Name']
    else:
        # If some Project IDs are missing, fill them with Project Name
        supply_df['Project ID'] = supply_df['Project ID'].fillna(supply_df['Project Name'])

    # If needed, derive "Project Category" from chunk size
    if 'Project Category' not in supply_df.columns:
        bins = [0,350000,500000,800000,1000000,float('inf')]
        labels= ['0-350k','350k-500k','500k-800k','800k-1M','1M+']
        supply_df['Project Category'] = pd.cut(
            supply_df['Est Cash Amount'], bins=bins, labels=labels
        )

    # Rename "State" -> "Project State" if needed
    if 'State' in supply_df.columns:
        supply_df.rename(columns={'State':'Project State'}, inplace=True)

    # ---------------------------------------------------------------------
    # Ensure Priority is numeric and sort the DataFrame
    # ---------------------------------------------------------------------
    if 'Priority' not in supply_df.columns:
        raise ValueError("Supply data must contain a 'Priority' column.")

    # Convert Priority to numeric
    supply_df['Priority'] = pd.to_numeric(supply_df['Priority'], errors='coerce')
    supply_df = supply_df.dropna(subset=['Priority'])  # Remove rows with invalid Priority

    # Sort by Priority ASC, GDF Profit DESC, Est Cash Amount DESC, Project_date ASC
    supply_df = supply_df.sort_values(
        by=['Priority', 'GDF Profit', 'Est Cash Amount', 'Project_date'],
        ascending=[True, False, False, True]
    )

    # Keep an original copy of Est Cash Amount for reference
    supply_df['Original Est Cash Amount'] = supply_df['Est Cash Amount']

    return supply_df

# -------------------------------------------------------------------------
# 2. Fetch Demand
# -------------------------------------------------------------------------
def fetch_demand():
    ws = sheet.worksheet("Demand")
    data = ws.get_all_values()
    demand_df = pd.DataFrame(data[1:], columns=data[0])

    # Rename columns if needed
    demand_df = demand_df.rename(columns={'purchaser_name': 'Purchaser Name'})

    # Convert numeric
    demand_df['approximated_appetite'] = pd.to_numeric(
        demand_df['approximated_appetite'].replace({',':''}, regex=True)
    )
    # Remove 'pending_amount' usage if it exists
    if 'pending_amount' in demand_df.columns:
        demand_df.drop(columns=['pending_amount'], inplace=True)

    # Convert date
    demand_df['Purchaser_Date'] = pd.to_datetime(demand_df['Purchaser_Date'], errors='coerce')
    demand_df = demand_df.dropna(subset=['approximated_appetite','Purchaser_Date'])

    # Track remaining appetite
    demand_df['remaining_appetite'] = demand_df['approximated_appetite']

    # Rename "State" -> "Purchaser State" if needed
    if 'State' in demand_df.columns:
        demand_df.rename(columns={'State':'Purchaser State'}, inplace=True)

    # Ensure we have a "Purchaser Category"
    if 'Purchaser Category' not in demand_df.columns:
        bins = [0,350000,500000,800000,1000000,float('inf')]
        labels= ['0-350k','350k-500k','500k-800k','800k-1M','1M+']
        demand_df['Purchaser Category'] = pd.cut(
            demand_df['approximated_appetite'], bins=bins, labels=labels
        )

    # Add priority for state (CA=1, others=2)
    demand_df['State Priority'] = demand_df['Purchaser State'].apply(
        lambda x: 1 if x == 'CA' else 2
    )

    # (Optional) Sort demand initially by appetite desc, then state priority
    demand_df = demand_df.sort_values(
        by=['approximated_appetite', 'State Priority'],
        ascending=[False, True]
    )

    return demand_df

# -------------------------------------------------------------------------
# 3. Allocate *per-chunk*, no partial chunk, no partial project,
#    Bank bridging only if we find a single future purchaser to exit.
# -------------------------------------------------------------------------
def allocate_chunks_no_partial(
    supply_df: pd.DataFrame,
    demand_df: pd.DataFrame,
    bank_credit_limit: float = 10_000_000
):
    """
    Steps:
      1) Group supply by Project Name. Each project can have multiple "chunks".
      2) Each chunk is allocated fully (no partial chunk) to either:
         - A direct purchaser with date <= project_date
         - Or bank bridging if we can find exactly one future purchaser
           (with date >= project_date) who has enough appetite for the chunk.
           We create a Bank Entry + Bank Exit immediately.
      3) If any chunk fails, we revert the entire project (no partial project).
      4) We build:
         - "Allocations" DataFrame: direct purchases, bank entries, and bank exits
         - "Financing Chunks" DataFrame: overview of financed chunks
         - "Summary" DataFrame: final result metrics
         - "Supply Summary" DataFrame with gross profit metrics
         - "SupplyAfter" DataFrame (includes 'Result' column)
         - "DemandAfter" DataFrame with final states
    """
    supply_df = supply_df.copy()
    demand_df = demand_df.copy()

    # Convert to dict records for manipulation
    supply_records = supply_df.to_dict('records')
    demand_records = demand_df.to_dict('records')

    allocations = []      # final ledger of all lines
    financed_chunks = []  # store chunk-level financing info
    bank_available = bank_credit_limit

    allocated_projects = set()  # to track fully allocated projects

    # ---------------------------------------------------------------------
    # Helper: find a direct purchaser
    # ---------------------------------------------------------------------
    def find_direct_purchaser(project_date, chunk_cost, d_records):
        """
        Return a single purchaser dict with date <= project_date
        and remaining_appetite >= chunk_cost, picking the one
        with the largest appetite.
        If found, return (purchaser, index_in_d_records). Else None.
        """
        eligible = [
            (p, i) for i, p in enumerate(d_records)
            if (p['Purchaser_Date'] <= project_date) and (p['remaining_appetite'] >= chunk_cost)
        ]
        if not eligible:
            return None
        # Sort by largest appetite
        eligible.sort(key=lambda x: x[0]['remaining_appetite'], reverse=True)
        return eligible[0]  # top (p, i)

    # ---------------------------------------------------------------------
    # Helper: find a future purchaser for bank bridging
    # ---------------------------------------------------------------------
    def find_future_purchaser(project_date, chunk_cost, d_records):
        """
        Return a single future purchaser who can buy the entire chunk,
        i.e. p['Purchaser_Date'] >= project_date and p['remaining_appetite'] >= chunk_cost.
        Pick the largest appetite if multiple.
        """
        future = [
            (p, i) for i, p in enumerate(d_records)
            if (p['Purchaser_Date'] >= project_date) and (p['remaining_appetite'] >= chunk_cost)
        ]
        if not future:
            return None
        future.sort(key=lambda x: x[0]['remaining_appetite'], reverse=True)
        return future[0]

    # ---------------------------------------------------------------------
    # Group supply by project name
    # ---------------------------------------------------------------------
    by_project = defaultdict(list)
    for r in supply_records:
        pname = r['Project Name']
        by_project[pname].append(r)

    # Build project list with total cost, earliest date, and priority
    projects = []
    for pname, recs in by_project.items():
        total_cost = sum(rr['Est Cash Amount'] for rr in recs)
        proj_date = min(r['Project_date'] for r in recs)
        proj_priority = min(r['Priority'] for r in recs)  # min Priority among its chunks
        proj_gdf_profit = max(r['GDF Profit'] for r in recs)
        projects.append((pname, recs, total_cost, proj_date, proj_priority, proj_gdf_profit))

    # Sort projects by Priority ASC, then GDF Profit DESC, then total_cost DESC
    projects.sort(key=lambda x: (x[4], -x[5], -x[2]))

    # ---------------------------------------------------------------------
    # Allocate each project (no partial project)
    # ---------------------------------------------------------------------
    for (pname, recs, total_cost, pdate, proj_priority, proj_gdf_profit) in projects:
        temp_allocs = []
        temp_financed = []

        # Snapshot of demand + local bank for tries
        local_demand = [dict(d) for d in demand_records]
        local_bank = bank_available

        # Sort chunks within the project (by priority asc, then cost desc)
        sorted_chunks = sorted(recs, key=lambda x: (x['Priority'], -x['Est Cash Amount']))

        project_ok = True  # track if project can be fully allocated

        for ch in sorted_chunks:
            chunk_cost = ch['Est Cash Amount']
            p_name = ch['Project Name']
            p_cat  = ch.get('Project Category','')
            p_st   = ch.get('Project State','')
            p_gdf  = ch['GDF Profit']
            p_id   = ch['Project ID']
            p_priority = ch['Priority']
            p_chunk_date = ch['Project_date']

            # 1) Try direct purchaser
            direct = find_direct_purchaser(p_chunk_date, chunk_cost, local_demand)
            if direct:
                pur, pur_idx = direct
                pur['remaining_appetite'] -= chunk_cost

                # Record direct purchase
                temp_allocs.append({
                    "Purchaser Name": pur['Purchaser Name'],
                    "Purchaser State": pur.get('Purchaser State',''),
                    "Purchaser Date": pur['Purchaser_Date'],
                    "Purchaser Category": pur.get('Purchaser Category',''),

                    "Project Name": p_name,
                    "Project Date": p_chunk_date,
                    "Project Category": p_cat,
                    "Project State": p_st,
                    "GDF Profit": p_gdf,
                    "Priority": p_priority,

                    "Allocated Amount": chunk_cost,
                    "Financed Amount": 0.0,  # no bank involvement
                    "Remaining Appetite After": pur['remaining_appetite'],
                    "Original Appetite": pur['approximated_appetite'],

                    "Project ID": p_id,
                    "Result": True  # tentatively true
                })
                continue

            # 2) Bank bridging
            if local_bank >= chunk_cost:
                future = find_future_purchaser(p_chunk_date, chunk_cost, local_demand)
                if future:
                    f_pur, f_idx = future
                    # Bank invests now
                    local_bank -= chunk_cost

                    # (a) Bank Entry
                    temp_allocs.append({
                        "Purchaser Name": "Bank Credit",
                        "Purchaser State": "N/A",
                        "Purchaser Date": p_chunk_date,
                        "Purchaser Category": "N/A",

                        "Project Name": p_name,
                        "Project Date": p_chunk_date,
                        "Project Category": p_cat,
                        "Project State": p_st,
                        "GDF Profit": p_gdf,
                        "Priority": p_priority,

                        "Allocated Amount": chunk_cost,
                        "Financed Amount": chunk_cost,  # bank invests
                        "Remaining Appetite After": 0,  # bank has no "appetite"
                        "Original Appetite": 0,

                        "Project ID": p_id,
                        "Result": True
                    })

                    # (b) Bank Exit => future purchaser
                    f_pur['remaining_appetite'] -= chunk_cost
                    local_bank += chunk_cost  # bank recovers the chunk cost
                    exit_date = f_pur['Purchaser_Date']

                    temp_allocs.append({
                        "Purchaser Name": f_pur['Purchaser Name'],
                        "Purchaser State": f_pur.get('Purchaser State',''),
                        "Purchaser Date": exit_date,
                        "Purchaser Category": f_pur.get('Purchaser Category',''),

                        "Project Name": p_name,
                        "Project Date": p_chunk_date,  # bridging date
                        "Project Category": p_cat,
                        "Project State": p_st,
                        "GDF Profit": p_gdf,
                        "Priority": p_priority,

                        "Allocated Amount": chunk_cost,
                        "Financed Amount": -chunk_cost,  # negative => bank exit
                        "Remaining Appetite After": f_pur['remaining_appetite'],
                        "Original Appetite": f_pur['approximated_appetite'],

                        "Project ID": p_id,
                        "Result": True
                    })

                    # Record bridging in financed_chunks
                    temp_financed.append({
                        "Project ID": p_id,
                        "Project Name": p_name,
                        "Chunk Cost": chunk_cost,
                        "Purchase Date": p_chunk_date,
                        "Exit Date": exit_date,
                        "Exit Purchaser Name": f_pur['Purchaser Name'],
                        "Remaining Appetite After": f_pur['remaining_appetite'],
                        "Original Appetite": f_pur['approximated_appetite']
                    })
                else:
                    # No single future purchaser => fail
                    project_ok = False
                    break
            else:
                # Bank doesn't have enough
                project_ok = False
                break

        # After attempting all chunks in this project
        if project_ok:
            # project fully allocated => mark it
            allocated_projects.add(pname)
            demand_records = local_demand
            bank_available = local_bank
            allocations.extend(temp_allocs)
            financed_chunks.extend(temp_financed)
        else:
            # revert => none of these chunks get allocated
            for ch in recs:
                allocations.append({
                    "Purchaser Name": None,
                    "Purchaser State": None,
                    "Purchaser Date": None,
                    "Purchaser Category": None,

                    "Project Name": ch['Project Name'],
                    "Project Date": ch['Project_date'],
                    "Project Category": ch.get('Project Category',''),
                    "Project State": ch.get('Project State',''),
                    "GDF Profit": ch['GDF Profit'],
                    "Priority": ch['Priority'],

                    "Allocated Amount": 0.0,
                    "Financed Amount": 0.0,
                    "Remaining Appetite After": None,
                    "Original Appetite": None,

                    "Project ID": ch['Project ID'],
                    "Result": False
                })

    # ---------------------------------------------------------------------
    # Convert to DataFrame
    # ---------------------------------------------------------------------
    allocations_df = pd.DataFrame(allocations)
    # Reorder columns for Allocations
    alloc_cols = [
        "Purchaser Name","Purchaser State","Purchaser Date","Purchaser Category",
        "Project Name","Project Date","Project Category","Project State","GDF Profit",
        "Priority",
        "Allocated Amount","Financed Amount","Remaining Appetite After",
        "Original Appetite","Project ID","Result"
    ]
    alloc_cols = [c for c in alloc_cols if c in allocations_df.columns]
    allocations_df = allocations_df[alloc_cols]

    # Financing DF
    financed_df = pd.DataFrame(financed_chunks)
    fin_cols = [
        "Project ID","Project Name","Chunk Cost","Purchase Date","Exit Date",
        "Exit Purchaser Name","Remaining Appetite After","Original Appetite"
    ]
    for cc in fin_cols:
        if cc not in financed_df.columns:
            financed_df[cc] = None
    financed_df = financed_df[fin_cols]

    # ---------------------------------------------------------------------
    # Summary Metrics
    # ---------------------------------------------------------------------
    total_alloc = allocations_df['Allocated Amount'].sum()
    financed_used = allocations_df.loc[allocations_df['Financed Amount'] > 0, 'Financed Amount'].sum()
    bank_remaining = bank_available

    # Demand sums
    original_total_appetite = demand_df['approximated_appetite'].sum()

    # Convert final demand_records to DataFrame
    demand_records_df = pd.DataFrame(demand_records)
    remaining_demand = demand_records_df['remaining_appetite'].sum()

    # Supply sums
    original_total_project_value = supply_df['Original Est Cash Amount'].sum()
    unallocated_projects = set(by_project.keys()) - allocated_projects
    unallocated_supply_df = supply_df[supply_df['Project Name'].isin(unallocated_projects)]
    remaining_supply_value = unallocated_supply_df['Est Cash Amount'].sum()

    pct_alloc = 0.0
    if original_total_project_value > 0:
        pct_alloc = (total_alloc / original_total_project_value) * 100

    # Build Summary
    summary_data = [
        {"Metric": "--- RESULTS SUMMARY ---", "Value": ""},
        {"Metric": "Total Allocated", "Value": f"{total_alloc:,.2f}"},
        {"Metric": "Financed (Bank Used)", "Value": f"{financed_used:,.2f}"},
        {"Metric": "Bank Credit Remaining", "Value": f"{bank_remaining:,.2f}"},

        {"Metric": "", "Value": ""},
        {"Metric": "DEMAND SUM", "Value": ""},
        {"Metric": "Original Total Appetite", "Value": f"{original_total_appetite:,.2f}"},
        {"Metric": "Remaining Demand", "Value": f"{remaining_demand:,.2f}"},

        {"Metric": "", "Value": ""},
        {"Metric": "% of Allocated Supply", "Value": f"{pct_alloc:.2f}%"},

        {"Metric": "", "Value": ""},
        {"Metric": "SUPPLY SUM", "Value": ""},
        {"Metric": "Original Total Project Value", "Value": f"{original_total_project_value:,.2f}"},
        {"Metric": "Remaining Supply", "Value": f"{remaining_supply_value:,.2f}"},

        {"Metric": "", "Value": ""},
        {"Metric": "--- GROSS PROFIT METRICS ---", "Value": ""},
        {"Metric": "Total Gross Profit (all projects)", "Value": ""},
        {"Metric": "Total Gross Profit Allocated", "Value": ""},
        {"Metric": "Total Gross Profit Unallocated", "Value": ""}
    ]
    summary_df = pd.DataFrame(summary_data)

    # ---------------------------------------------------------------------
    # 5. Create Supply Summary with Gross Profit Metrics
    # ---------------------------------------------------------------------
    # Merge the final 'Result' from the allocations into supply_df
    project_results = allocations_df.groupby('Project Name')['Result'].all().reset_index()
    project_results.rename(columns={'Result': 'Result'}, inplace=True)

    # Merge with supply_df to add chunk-level 'Result'
    supply_after_df = supply_df.merge(project_results, on='Project Name', how='left')
    # For projects not in allocations, set 'Result' = False
    supply_after_df['Result'] = supply_after_df['Result'].fillna(False)

    # Now build supply_summary
    supply_summary = supply_after_df.groupby('Project Name').agg(
        Total_Chunks_Before=('Est Cash Amount', 'count'),
        # sum of booleans => # allocated
        Total_Chunks_After=('Result', lambda x: x.sum()),
        Total_Project_Value=('Est Cash Amount', 'sum'),
        Latest_Project_Date=('Project_date', 'max'),
        Max_Est_Cash_Amount=('Est Cash Amount', 'max'),
        GDF_Profit=('GDF Profit', 'max'),
        Priority=('Priority', 'max')
    ).reset_index()

    # Compute Gross_Profit
    supply_summary['Gross_Profit'] = supply_summary['Total_Project_Value'] * supply_summary['GDF_Profit']

    # Gross Profit metrics
    total_gross_profit = supply_summary['Gross_Profit'].sum()
    total_gross_profit_allocated = supply_summary.loc[supply_summary['Total_Chunks_After'] > 0, 'Gross_Profit'].sum()
    total_gross_profit_unallocated = supply_summary.loc[supply_summary['Total_Chunks_After'] == 0, 'Gross_Profit'].sum()

    # Update the last rows of summary_df with these metrics
    summary_df.iloc[-3, summary_df.columns.get_loc("Value")] = f"{total_gross_profit:,.2f}"
    summary_df.iloc[-2, summary_df.columns.get_loc("Value")] = f"{total_gross_profit_allocated:,.2f}"
    summary_df.iloc[-1, summary_df.columns.get_loc("Value")] = f"{total_gross_profit_unallocated:,.2f}"

    # ---------------------------------------------------------------------
    # 6. Build DemandAfter DataFrame
    # ---------------------------------------------------------------------
    updated_demand_df = demand_records_df.copy()
    # allocated_amount = original appetite - remaining
    updated_demand_df['allocated_amount'] = (
        updated_demand_df['approximated_appetite'] - updated_demand_df['remaining_appetite']
    )

    # Create the Financing column as 0.0 by default before merging
    updated_demand_df["Financing"] = 0.0

    # If we have any negative Financed Amount (i.e. bank exit),
    # sum them up by Purchaser Name (convert negative to positive).
    if not allocations_df.empty and "Financed Amount" in allocations_df.columns:
        financing_by_purchaser = (
            allocations_df[allocations_df['Financed Amount'] < 0]
            .groupby('Purchaser Name')['Financed Amount'].sum()
            .abs()
            .reset_index(name='Financing')
        )
        # Merge into updated_demand_df
        updated_demand_df = updated_demand_df.merge(
            financing_by_purchaser, on='Purchaser Name', how='left', suffixes=('', '_calc')
        )
        # Where merging found a value, overwrite 0.0
        updated_demand_df['Financing'] = updated_demand_df['Financing_calc'].fillna(0)
        updated_demand_df.drop(columns=['Financing_calc'], inplace=True)

    # Reorder columns for DemandAfter
    demand_after_cols = [
        "Purchaser Name",
        "approximated_appetite",
        "allocated_amount",
        "Purchaser State",
        "Purchaser_Date",
        "Financing",
        "remaining_appetite",
        "Purchaser Category",
        "State Priority"
    ]

    for c in demand_after_cols:
        if c not in updated_demand_df.columns:
            updated_demand_df[c] = None

    updated_demand_df = updated_demand_df[demand_after_cols]

    # ---------------------------------------------------------------------
    # Return all DataFrames
    # ---------------------------------------------------------------------
    return (
        allocations_df,
        financed_df,
        summary_df,
        supply_summary,
        supply_after_df,
        updated_demand_df
    )

# -------------------------------------------------------------------------
# 9. Main Execution
# -------------------------------------------------------------------------
def main():
    supply_df = fetch_supply()
    demand_df = fetch_demand()

    (
        allocations_df,
        financed_df,
        summary_df,
        supply_summary,
        updated_supply,
        updated_demand_df
    ) = allocate_chunks_no_partial(supply_df, demand_df, bank_credit_limit=10_000_000)

    # Print summary to console
    print(summary_df.to_string(index=False))

    # Save to Excel with multiple tabs
    with pd.ExcelWriter("allocation_results.xlsx", engine="xlsxwriter") as writer:
        allocations_df.to_excel(writer, sheet_name="Allocations", index=False)
        financed_df.to_excel(writer, sheet_name="Financing Chunks", index=False)
        summary_df.to_excel(writer, sheet_name="Summary", index=False)
        supply_summary.to_excel(writer, sheet_name="Supply Summary", index=False)
        updated_supply.to_excel(writer, sheet_name="SupplyAfter", index=False)
        updated_demand_df.to_excel(writer, sheet_name="DemandAfter", index=False)

    print("\nResults saved to 'allocation_results.xlsx' with:")
    print(" - Allocations (all direct + bank lines)")
    print(" - Financing Chunks (bank bridging details)")
    print(" - Summary (your result metrics including Gross Profit)")
    print(" - Supply Summary (detailed project allocations and Gross Profit)")
    print(" - SupplyAfter (with 'Result' column)")
    print(" - DemandAfter (final states)")

if __name__ == "__main__":
    main()


                           Metric          Value
          --- RESULTS SUMMARY ---               
                  Total Allocated  33,985,664.47
             Financed (Bank Used)           0.00
            Bank Credit Remaining  10,000,000.00
                                                
                       DEMAND SUM               
          Original Total Appetite 100,414,628.00
                 Remaining Demand  66,428,963.53
                                                
            % of Allocated Supply         41.35%
                                                
                       SUPPLY SUM               
     Original Total Project Value  82,199,692.42
                 Remaining Supply  48,214,027.95
                                                
     --- GROSS PROFIT METRICS ---               
Total Gross Profit (all projects)     202,573.93
     Total Gross Profit Allocated      87,859.01
   Total Gross Profit Unallocated     114,714.93

Results saved to 'a

# **Gross Total / Cost of Interest / Net Profit metrics of bank financing**

In [15]:
import gspread
from google.colab import auth
from google.auth import default
import pandas as pd
import numpy as np
from collections import defaultdict

# -------------------------------------------------------------------------
# 0. Authenticate and open the Google Sheet
# -------------------------------------------------------------------------
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

title = "2025 Project Agenda"
sheet = gc.open(title)

# -------------------------------------------------------------------------
# 1. Fetch Supply
# -------------------------------------------------------------------------
def fetch_supply():
    ws = sheet.worksheet("Supply")
    data = ws.get_all_values()
    supply_df = pd.DataFrame(data[1:], columns=data[0])

    # Keep only rows marked 'Y'
    supply_df = supply_df[supply_df['Take into consideration?'].str.lower() == 'y']

    # Convert numeric fields
    supply_df['Est Cash Amount'] = pd.to_numeric(
        supply_df['Est Cash Amount'].replace({',': ''}, regex=True)
    )
    supply_df['Project_date'] = pd.to_datetime(supply_df['Project_date'], errors='coerce')

    # Convert GDF Profit to numeric (e.g., "25%" -> 0.25)
    supply_df['GDF Profit'] = (
        supply_df['GDF Profit'].str.rstrip('%').astype(float) / 100
    )

    # Drop rows missing critical columns
    supply_df = supply_df.dropna(subset=['Est Cash Amount','Project_date','GDF Profit'])

    # Ensure we have a Project ID
    if 'Project ID' not in supply_df.columns or supply_df['Project ID'].isnull().all():
        supply_df['Project ID'] = supply_df['Project Name']
    else:
        supply_df['Project ID'] = supply_df['Project ID'].fillna(supply_df['Project Name'])

    # If needed, derive "Project Category" from chunk size
    if 'Project Category' not in supply_df.columns:
        bins = [0,350000,500000,800000,1000000,float('inf')]
        labels= ['0-350k','350k-500k','500k-800k','800k-1M','1M+']
        supply_df['Project Category'] = pd.cut(
            supply_df['Est Cash Amount'], bins=bins, labels=labels
        )

    # Rename "State" -> "Project State" if needed
    if 'State' in supply_df.columns:
        supply_df.rename(columns={'State':'Project State'}, inplace=True)

    # Ensure Priority is numeric and sort
    if 'Priority' not in supply_df.columns:
        raise ValueError("Supply data must contain a 'Priority' column.")

    supply_df['Priority'] = pd.to_numeric(supply_df['Priority'], errors='coerce')
    supply_df = supply_df.dropna(subset=['Priority'])

    # Sort
    supply_df = supply_df.sort_values(
        by=['Priority', 'GDF Profit', 'Est Cash Amount', 'Project_date'],
        ascending=[True, False, False, True]
    )

    # Keep an original copy of Est Cash Amount
    supply_df['Original Est Cash Amount'] = supply_df['Est Cash Amount']

    return supply_df

# -------------------------------------------------------------------------
# 2. Fetch Demand
# -------------------------------------------------------------------------
def fetch_demand():
    ws = sheet.worksheet("Demand")
    data = ws.get_all_values()
    demand_df = pd.DataFrame(data[1:], columns=data[0])

    # Rename columns if needed
    demand_df = demand_df.rename(columns={'purchaser_name': 'Purchaser Name'})

    # Convert numeric
    demand_df['approximated_appetite'] = pd.to_numeric(
        demand_df['approximated_appetite'].replace({',':''}, regex=True)
    )
    # Remove 'pending_amount' if it exists
    if 'pending_amount' in demand_df.columns:
        demand_df.drop(columns=['pending_amount'], inplace=True)

    # Convert dates
    demand_df['Purchaser_Date'] = pd.to_datetime(demand_df['Purchaser_Date'], errors='coerce')
    demand_df = demand_df.dropna(subset=['approximated_appetite','Purchaser_Date'])

    # Track remaining appetite
    demand_df['remaining_appetite'] = demand_df['approximated_appetite']

    # Rename "State" -> "Purchaser State" if needed
    if 'State' in demand_df.columns:
        demand_df.rename(columns={'State':'Purchaser State'}, inplace=True)

    # Ensure "Purchaser Category"
    if 'Purchaser Category' not in demand_df.columns:
        bins = [0,350000,500000,800000,1000000,float('inf')]
        labels= ['0-350k','350k-500k','500k-800k','800k-1M','1M+']
        demand_df['Purchaser Category'] = pd.cut(
            demand_df['approximated_appetite'], bins=bins, labels=labels
        )

    # State Priority: CA=1, others=2
    demand_df['State Priority'] = demand_df['Purchaser State'].apply(
        lambda x: 1 if x == 'CA' else 2
    )

    # Sort demand by appetite desc, then state priority
    demand_df = demand_df.sort_values(
        by=['approximated_appetite', 'State Priority'],
        ascending=[False, True]
    )

    return demand_df

# -------------------------------------------------------------------------
# 3. Allocate function
# -------------------------------------------------------------------------
def allocate_chunks_no_partial(
    supply_df: pd.DataFrame,
    demand_df: pd.DataFrame,
    bank_credit_limit: float = 0_000_000
):
    supply_df = supply_df.copy()
    demand_df = demand_df.copy()

    supply_records = supply_df.to_dict('records')
    demand_records = demand_df.to_dict('records')

    allocations = []
    financed_chunks = []
    bank_available = bank_credit_limit

    allocated_projects = set()

    def find_direct_purchaser(project_date, chunk_cost, d_records):
        eligible = [
            (p, i) for i, p in enumerate(d_records)
            if (p['Purchaser_Date'] <= project_date) and (p['remaining_appetite'] >= chunk_cost)
        ]
        if not eligible:
            return None
        eligible.sort(key=lambda x: x[0]['remaining_appetite'], reverse=True)
        return eligible[0]

    def find_future_purchaser(project_date, chunk_cost, d_records):
        future = [
            (p, i) for i, p in enumerate(d_records)
            if (p['Purchaser_Date'] >= project_date) and (p['remaining_appetite'] >= chunk_cost)
        ]
        if not future:
            return None
        future.sort(key=lambda x: x[0]['remaining_appetite'], reverse=True)
        return future[0]

    # Group supply by project
    from collections import defaultdict
    by_project = defaultdict(list)
    for r in supply_records:
        pname = r['Project Name']
        by_project[pname].append(r)

    projects = []
    for pname, recs in by_project.items():
        total_cost = sum(rr['Est Cash Amount'] for rr in recs)
        proj_date = min(r['Project_date'] for r in recs)
        proj_priority = min(r['Priority'] for r in recs)
        proj_gdf_profit = max(r['GDF Profit'] for r in recs)
        projects.append((pname, recs, total_cost, proj_date, proj_priority, proj_gdf_profit))

    # Sort projects
    projects.sort(key=lambda x: (x[4], -x[5], -x[2]))

    for (pname, recs, total_cost, pdate, proj_priority, proj_gdf_profit) in projects:
        temp_allocs = []
        temp_financed = []

        local_demand = [dict(d) for d in demand_records]
        local_bank = bank_available

        sorted_chunks = sorted(recs, key=lambda x: (x['Priority'], -x['Est Cash Amount']))

        project_ok = True

        for ch in sorted_chunks:
            chunk_cost = ch['Est Cash Amount']
            p_name = ch['Project Name']
            p_cat  = ch.get('Project Category','')
            p_st   = ch.get('Project State','')
            p_gdf  = ch['GDF Profit']
            p_id   = ch['Project ID']
            p_priority = ch['Priority']
            p_chunk_date = ch['Project_date']

            # Direct purchaser
            direct = find_direct_purchaser(p_chunk_date, chunk_cost, local_demand)
            if direct:
                pur, pur_idx = direct
                pur['remaining_appetite'] -= chunk_cost
                temp_allocs.append({
                    "Purchaser Name": pur['Purchaser Name'],
                    "Purchaser State": pur.get('Purchaser State',''),
                    "Purchaser Date": pur['Purchaser_Date'],
                    "Purchaser Category": pur.get('Purchaser Category',''),

                    "Project Name": p_name,
                    "Project Date": p_chunk_date,
                    "Project Category": p_cat,
                    "Project State": p_st,
                    "GDF Profit": p_gdf,
                    "Priority": p_priority,

                    "Allocated Amount": chunk_cost,
                    "Financed Amount": 0.0,
                    "Remaining Appetite After": pur['remaining_appetite'],
                    "Original Appetite": pur['approximated_appetite'],

                    "Project ID": p_id,
                    "Result": True
                })
                continue

            # Bank bridging
            if local_bank >= chunk_cost:
                future = find_future_purchaser(p_chunk_date, chunk_cost, local_demand)
                if future:
                    f_pur, f_idx = future
                    local_bank -= chunk_cost

                    # Bank Entry
                    temp_allocs.append({
                        "Purchaser Name": "Bank Credit",
                        "Purchaser State": "N/A",
                        "Purchaser Date": p_chunk_date,
                        "Purchaser Category": "N/A",

                        "Project Name": p_name,
                        "Project Date": p_chunk_date,
                        "Project Category": p_cat,
                        "Project State": p_st,
                        "GDF Profit": p_gdf,
                        "Priority": p_priority,

                        "Allocated Amount": chunk_cost,
                        "Financed Amount": chunk_cost,
                        "Remaining Appetite After": 0,
                        "Original Appetite": 0,

                        "Project ID": p_id,
                        "Result": True
                    })

                    # Bank Exit
                    f_pur['remaining_appetite'] -= chunk_cost
                    local_bank += chunk_cost
                    exit_date = f_pur['Purchaser_Date']

                    temp_allocs.append({
                        "Purchaser Name": f_pur['Purchaser Name'],
                        "Purchaser State": f_pur.get('Purchaser State',''),
                        "Purchaser Date": exit_date,
                        "Purchaser Category": f_pur.get('Purchaser Category',''),

                        "Project Name": p_name,
                        "Project Date": p_chunk_date,
                        "Project Category": p_cat,
                        "Project State": p_st,
                        "GDF Profit": p_gdf,
                        "Priority": p_priority,

                        "Allocated Amount": chunk_cost,
                        "Financed Amount": -chunk_cost,
                        "Remaining Appetite After": f_pur['remaining_appetite'],
                        "Original Appetite": f_pur['approximated_appetite'],

                        "Project ID": p_id,
                        "Result": True
                    })

                    # Record bridging in financed_chunks
                    temp_financed.append({
                        "Project ID": p_id,
                        "Project Name": p_name,
                        "Chunk Cost": chunk_cost,
                        "Purchase Date": p_chunk_date,
                        "Exit Date": exit_date,
                        "Exit Purchaser Name": f_pur['Purchaser Name'],
                        "Remaining Appetite After": f_pur['remaining_appetite'],
                        "Original Appetite": f_pur['approximated_appetite'],
                        "GDF Profit": p_gdf  # <-- Store GDF Profit for calculations
                    })
                else:
                    project_ok = False
                    break
            else:
                project_ok = False
                break

        if project_ok:
            allocated_projects.add(pname)
            demand_records = local_demand
            bank_available = local_bank
            allocations.extend(temp_allocs)
            financed_chunks.extend(temp_financed)
        else:
            # revert
            for ch in recs:
                allocations.append({
                    "Purchaser Name": None,
                    "Purchaser State": None,
                    "Purchaser Date": None,
                    "Purchaser Category": None,

                    "Project Name": ch['Project Name'],
                    "Project Date": ch['Project_date'],
                    "Project Category": ch.get('Project Category',''),
                    "Project State": ch.get('Project State',''),
                    "GDF Profit": ch['GDF Profit'],
                    "Priority": ch['Priority'],

                    "Allocated Amount": 0.0,
                    "Financed Amount": 0.0,
                    "Remaining Appetite After": None,
                    "Original Appetite": None,

                    "Project ID": ch['Project ID'],
                    "Result": False
                })

    # ---------------------------------------------------------------------
    # Build Allocations DF
    # ---------------------------------------------------------------------
    allocations_df = pd.DataFrame(allocations)
    alloc_cols = [
        "Purchaser Name","Purchaser State","Purchaser Date","Purchaser Category",
        "Project Name","Project Date","Project Category","Project State","GDF Profit",
        "Priority","Allocated Amount","Financed Amount","Remaining Appetite After",
        "Original Appetite","Project ID","Result"
    ]
    alloc_cols = [c for c in alloc_cols if c in allocations_df.columns]
    allocations_df = allocations_df[alloc_cols]

    # ---------------------------------------------------------------------
    # Build Financing DF
    # ---------------------------------------------------------------------
    financed_df = pd.DataFrame(financed_chunks)
    fin_cols = [
        "Project ID","Project Name","Chunk Cost","Purchase Date","Exit Date",
        "Exit Purchaser Name","Remaining Appetite After","Original Appetite",
        "GDF Profit"
    ]
    for cc in fin_cols:
        if cc not in financed_df.columns:
            financed_df[cc] = None
    financed_df = financed_df[fin_cols]

    # Now add the extra columns:
    #  Time of Bank Credit, Cost of Interest, Gross Profit, Net Profit, Net profit %
    # ---------------------------------------------------------------------
    if not financed_df.empty:
        # Convert to datetime if not already
        financed_df['Purchase Date'] = pd.to_datetime(financed_df['Purchase Date'], errors='coerce')
        financed_df['Exit Date'] = pd.to_datetime(financed_df['Exit Date'], errors='coerce')

        # Calculate months between Purchase and Exit
        financed_df['Time of Bank Credit'] = (
            (financed_df['Exit Date'] - financed_df['Purchase Date']).dt.days // 30
        ).fillna(0).astype(int)

        # 1% monthly interest (as implied by your example)
        financed_df['Cost of Interest'] = (
            financed_df['Chunk Cost'] * 0.01 * financed_df['Time of Bank Credit']
        )

        financed_df['Gross Profit'] = (
            financed_df['Chunk Cost'] * financed_df['GDF Profit']
        )

        financed_df['Net Profit'] = (
            financed_df['Gross Profit'] - financed_df['Cost of Interest']
        )

        financed_df['Net profit %'] = financed_df['Net Profit'] / financed_df['Chunk Cost']
    else:
        # If no bridging occurred, just ensure columns exist
        financed_df['Time of Bank Credit'] = None
        financed_df['Cost of Interest'] = None
        financed_df['Gross Profit'] = None
        financed_df['Net Profit'] = None
        financed_df['Net profit %'] = None

    # ---------------------------------------------------------------------
    # Summary Metrics
    # ---------------------------------------------------------------------
    total_alloc = allocations_df['Allocated Amount'].sum()
    financed_used = allocations_df.loc[allocations_df['Financed Amount'] > 0, 'Financed Amount'].sum()
    bank_remaining = bank_available

    original_total_appetite = demand_df['approximated_appetite'].sum()
    demand_records_df = pd.DataFrame(demand_records)
    remaining_demand = demand_records_df['remaining_appetite'].sum()

    original_total_project_value = supply_df['Original Est Cash Amount'].sum()
    unallocated_projects = set(by_project.keys()) - allocated_projects
    unallocated_supply_df = supply_df[supply_df['Project Name'].isin(unallocated_projects)]
    remaining_supply_value = unallocated_supply_df['Est Cash Amount'].sum()

    pct_alloc = 0.0
    if original_total_project_value > 0:
        pct_alloc = (total_alloc / original_total_project_value) * 100

    summary_data = [
        {"Metric": "--- RESULTS SUMMARY ---", "Value": ""},
        {"Metric": "Total Allocated", "Value": f"{total_alloc:,.2f}"},
        {"Metric": "Financed (Bank Used)", "Value": f"{financed_used:,.2f}"},
        {"Metric": "Bank Credit Remaining", "Value": f"{bank_remaining:,.2f}"},

        {"Metric": "", "Value": ""},
        {"Metric": "DEMAND SUM", "Value": ""},
        {"Metric": "Original Total Appetite", "Value": f"{original_total_appetite:,.2f}"},
        {"Metric": "Remaining Demand", "Value": f"{remaining_demand:,.2f}"},

        {"Metric": "", "Value": ""},
        {"Metric": "% of Allocated Supply", "Value": f"{pct_alloc:.2f}%"},

        {"Metric": "", "Value": ""},
        {"Metric": "SUPPLY SUM", "Value": ""},
        {"Metric": "Original Total Project Value", "Value": f"{original_total_project_value:,.2f}"},
        {"Metric": "Remaining Supply", "Value": f"{remaining_supply_value:,.2f}"},

        {"Metric": "", "Value": ""},
        {"Metric": "--- GROSS PROFIT METRICS ---", "Value": ""},
        {"Metric": "Total Gross Profit (all projects)", "Value": ""},
        {"Metric": "Total Gross Profit Allocated", "Value": ""},
        {"Metric": "Total Gross Profit Unallocated", "Value": ""}
    ]
    summary_df = pd.DataFrame(summary_data)

    # ---------------------------------------------------------------------
    # Supply Summary with Gross Profit Metrics
    # ---------------------------------------------------------------------
    project_results = allocations_df.groupby('Project Name')['Result'].all().reset_index()
    project_results.rename(columns={'Result': 'Result'}, inplace=True)
    supply_after_df = supply_df.merge(project_results, on='Project Name', how='left')
    supply_after_df['Result'] = supply_after_df['Result'].fillna(False)

    supply_summary = supply_after_df.groupby('Project Name').agg(
        Total_Chunks_Before=('Est Cash Amount', 'count'),
        Total_Chunks_After=('Result', lambda x: x.sum()),
        Total_Project_Value=('Est Cash Amount', 'sum'),
        Latest_Project_Date=('Project_date', 'max'),
        Max_Est_Cash_Amount=('Est Cash Amount', 'max'),
        GDF_Profit=('GDF Profit', 'max'),
        Priority=('Priority', 'max')
    ).reset_index()

    supply_summary['Gross_Profit'] = (
        supply_summary['Total_Project_Value'] * supply_summary['GDF_Profit']
    )

    total_gross_profit = supply_summary['Gross_Profit'].sum()
    total_gross_profit_allocated = supply_summary.loc[supply_summary['Total_Chunks_After'] > 0, 'Gross_Profit'].sum()
    total_gross_profit_unallocated = supply_summary.loc[supply_summary['Total_Chunks_After'] == 0, 'Gross_Profit'].sum()

    summary_df.iloc[-3, summary_df.columns.get_loc("Value")] = f"{total_gross_profit:,.2f}"
    summary_df.iloc[-2, summary_df.columns.get_loc("Value")] = f"{total_gross_profit_allocated:,.2f}"
    summary_df.iloc[-1, summary_df.columns.get_loc("Value")] = f"{total_gross_profit_unallocated:,.2f}"

    # ---------------------------------------------------------------------
    # Build DemandAfter
    # ---------------------------------------------------------------------
    updated_demand_df = demand_records_df.copy()
    updated_demand_df['allocated_amount'] = (
        updated_demand_df['approximated_appetite'] - updated_demand_df['remaining_appetite']
    )
    updated_demand_df["Financing"] = 0.0

    if not allocations_df.empty and "Financed Amount" in allocations_df.columns:
        financing_by_purchaser = (
            allocations_df[allocations_df['Financed Amount'] < 0]
            .groupby('Purchaser Name')['Financed Amount'].sum()
            .abs()
            .reset_index(name='Financing')
        )
        updated_demand_df = updated_demand_df.merge(
            financing_by_purchaser, on='Purchaser Name', how='left', suffixes=('', '_calc')
        )
        updated_demand_df['Financing'] = updated_demand_df['Financing_calc'].fillna(0)
        updated_demand_df.drop(columns=['Financing_calc'], inplace=True)

    demand_after_cols = [
        "Purchaser Name",
        "approximated_appetite",
        "allocated_amount",
        "Purchaser State",
        "Purchaser_Date",
        "Financing",
        "remaining_appetite",
        "Purchaser Category",
        "State Priority"
    ]
    for c in demand_after_cols:
        if c not in updated_demand_df.columns:
            updated_demand_df[c] = None

    updated_demand_df = updated_demand_df[demand_after_cols]

    return (
        allocations_df,
        financed_df,
        summary_df,
        supply_summary,
        supply_after_df,
        updated_demand_df
    )

# -------------------------------------------------------------------------
# 9. Main Execution
# -------------------------------------------------------------------------
def main():
    supply_df = fetch_supply()
    demand_df = fetch_demand()

    (
        allocations_df,
        financed_df,
        summary_df,
        supply_summary,
        updated_supply,
        updated_demand_df
    ) = allocate_chunks_no_partial(supply_df, demand_df, bank_credit_limit=0_000_000)

    # Print summary to console
    print(summary_df.to_string(index=False))

    # Save to Excel with multiple tabs
    with pd.ExcelWriter("allocation_results.xlsx", engine="xlsxwriter") as writer:
        allocations_df.to_excel(writer, sheet_name="Allocations", index=False)
        financed_df.to_excel(writer, sheet_name="Financing Chunks", index=False)
        summary_df.to_excel(writer, sheet_name="Summary", index=False)
        supply_summary.to_excel(writer, sheet_name="Supply Summary", index=False)
        updated_supply.to_excel(writer, sheet_name="SupplyAfter", index=False)
        updated_demand_df.to_excel(writer, sheet_name="DemandAfter", index=False)

    print("\nResults saved to 'allocation_results.xlsx' with:")
    print(" - Allocations (all direct + bank lines)")
    print(" - Financing Chunks (bank bridging details, now with interest & profit columns)")
    print(" - Summary (your result metrics including Gross Profit)")
    print(" - Supply Summary (detailed project allocations and Gross Profit)")
    print(" - SupplyAfter (with 'Result' column)")
    print(" - DemandAfter (final states)")

if __name__ == "__main__":
    main()


                           Metric          Value
          --- RESULTS SUMMARY ---               
                  Total Allocated  33,985,664.47
             Financed (Bank Used)           0.00
            Bank Credit Remaining           0.00
                                                
                       DEMAND SUM               
          Original Total Appetite 120,414,628.00
                 Remaining Demand  86,428,963.53
                                                
            % of Allocated Supply         41.35%
                                                
                       SUPPLY SUM               
     Original Total Project Value  82,199,692.42
                 Remaining Supply  48,214,027.95
                                                
     --- GROSS PROFIT METRICS ---               
Total Gross Profit (all projects)  20,257,393.30
     Total Gross Profit Allocated   8,785,900.77
   Total Gross Profit Unallocated  11,471,492.52

Results saved to 'a

**edits to match full allocation when the financing is 0**

In [18]:
import gspread
from google.colab import auth
from google.auth import default
import pandas as pd
import numpy as np
from collections import defaultdict

# -------------------------------------------------------------------------
# 0. Authenticate and open the Google Sheet
# -------------------------------------------------------------------------
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

title = "2025 Project Agenda"
sheet = gc.open(title)

# -------------------------------------------------------------------------
# 1. Fetch Supply
# -------------------------------------------------------------------------
def fetch_supply():
    ws = sheet.worksheet("Supply")
    data = ws.get_all_values()
    supply_df = pd.DataFrame(data[1:], columns=data[0])

    # Keep only rows marked 'Y'
    supply_df = supply_df[supply_df['Take into consideration?'].str.lower() == 'y']

    # Convert numeric fields
    supply_df['Est Cash Amount'] = pd.to_numeric(
        supply_df['Est Cash Amount'].replace({',': ''}, regex=True)
    )
    supply_df['Project_date'] = pd.to_datetime(supply_df['Project_date'], errors='coerce')

    # Convert GDF Profit to numeric (e.g., "25%" -> 0.25)
    supply_df['GDF Profit'] = (
        supply_df['GDF Profit'].str.rstrip('%').astype(float) / 100
    )

    # Drop rows missing critical columns
    supply_df = supply_df.dropna(subset=['Est Cash Amount','Project_date','GDF Profit'])

    # Ensure we have a Project ID
    if 'Project ID' not in supply_df.columns or supply_df['Project ID'].isnull().all():
        supply_df['Project ID'] = supply_df['Project Name']
    else:
        supply_df['Project ID'] = supply_df['Project ID'].fillna(supply_df['Project Name'])

    # If needed, derive "Project Category" from chunk size
    if 'Project Category' not in supply_df.columns:
        bins = [0, 350000, 500000, 800000, 1000000, float('inf')]
        labels= ['0-350k','350k-500k','500k-800k','800k-1M','1M+']
        supply_df['Project Category'] = pd.cut(
            supply_df['Est Cash Amount'], bins=bins, labels=labels
        )

    # Rename "State" -> "Project State" if needed
    if 'State' in supply_df.columns:
        supply_df.rename(columns={'State':'Project State'}, inplace=True)

    # Ensure Priority is numeric and sort
    if 'Priority' not in supply_df.columns:
        raise ValueError("Supply data must contain a 'Priority' column.")

    supply_df['Priority'] = pd.to_numeric(supply_df['Priority'], errors='coerce')
    supply_df = supply_df.dropna(subset=['Priority'])

    # Sort supply by (Priority ASC, GDF Profit DESC, Est Cash DESC, Project_date ASC)
    supply_df = supply_df.sort_values(
        by=['Priority', 'GDF Profit', 'Est Cash Amount', 'Project_date'],
        ascending=[True, False, False, True]
    )

    # Keep an original copy of Est Cash Amount
    supply_df['Original Est Cash Amount'] = supply_df['Est Cash Amount']

    return supply_df

# -------------------------------------------------------------------------
# 2. Fetch Demand
# -------------------------------------------------------------------------
def fetch_demand():
    ws = sheet.worksheet("Demand")
    data = ws.get_all_values()
    demand_df = pd.DataFrame(data[1:], columns=data[0])

    # Rename columns if needed
    demand_df = demand_df.rename(columns={'purchaser_name': 'Purchaser Name'})

    # Convert numeric
    demand_df['approximated_appetite'] = pd.to_numeric(
        demand_df['approximated_appetite'].replace({',':''}, regex=True)
    )
    # Remove 'pending_amount' if it exists
    if 'pending_amount' in demand_df.columns:
        demand_df.drop(columns=['pending_amount'], inplace=True)

    # Convert dates
    demand_df['Purchaser_Date'] = pd.to_datetime(demand_df['Purchaser_Date'], errors='coerce')
    demand_df = demand_df.dropna(subset=['approximated_appetite','Purchaser_Date'])

    # Track remaining appetite
    demand_df['remaining_appetite'] = demand_df['approximated_appetite']

    # Rename "State" -> "Purchaser State" if needed
    if 'State' in demand_df.columns:
        demand_df.rename(columns={'State':'Purchaser State'}, inplace=True)

    # Ensure "Purchaser Category"
    if 'Purchaser Category' not in demand_df.columns:
        bins = [0,350000,500000,800000,1000000,float('inf')]
        labels= ['0-350k','350k-500k','500k-800k','800k-1M','1M+']
        demand_df['Purchaser Category'] = pd.cut(
            demand_df['approximated_appetite'], bins=bins, labels=labels
        )

    # State Priority: CA=1, others=2
    demand_df['State Priority'] = demand_df['Purchaser State'].apply(
        lambda x: 1 if x == 'CA' else 2
    )

    # Sort demand by (approximated_appetite DESC, State Priority ASC)
    demand_df = demand_df.sort_values(
        by=['approximated_appetite', 'State Priority'],
        ascending=[False, True]
    )

    return demand_df

# -------------------------------------------------------------------------
# 3. Allocate function
# -------------------------------------------------------------------------
def allocate_chunks_no_partial(
    supply_df: pd.DataFrame,
    demand_df: pd.DataFrame,
    bank_credit_limit: float = 0_000_000
):
    """
    Allocates supply to demand.
    CA purchasers (State Priority=1) are prioritized over others (State Priority=2).
    If bank_credit_limit=0, bridging is disabled.
    All-or-nothing: if any chunk in a project fails to allocate, skip that project.
    """
    supply_df = supply_df.copy()
    demand_df = demand_df.copy()

    supply_records = supply_df.to_dict('records')
    demand_records = demand_df.to_dict('records')

    allocations = []
    financed_chunks = []
    bank_available = bank_credit_limit

    allocated_projects = set()

    # -----------------------------------------------
    # Updated: sort by State Priority, then appetite
    # -----------------------------------------------
    def find_direct_purchaser(project_date, chunk_cost, d_records):
        """
        Finds a direct purchaser who can buy now:
        - Must have Purchaser_Date <= project_date
        - Must have remaining_appetite >= chunk_cost
        Sort them by (State Priority ASC, remaining_appetite DESC).
        """
        eligible = [
            (p, i) for i, p in enumerate(d_records)
            if (p['Purchaser_Date'] <= project_date) and (p['remaining_appetite'] >= chunk_cost)
        ]
        if not eligible:
            return None

        # Sort by CA-first, then large appetite
        eligible.sort(key=lambda x: (x[0]['State Priority'], -x[0]['remaining_appetite']))
        return eligible[0]

    def find_future_purchaser(project_date, chunk_cost, d_records):
        """
        Finds a future purchaser for bank bridging:
        - Must have Purchaser_Date >= project_date
        - Must have remaining_appetite >= chunk_cost
        Sort them by (State Priority ASC, remaining_appetite DESC).
        """
        future = [
            (p, i) for i, p in enumerate(d_records)
            if (p['Purchaser_Date'] >= project_date) and (p['remaining_appetite'] >= chunk_cost)
        ]
        if not future:
            return None

        future.sort(key=lambda x: (x[0]['State Priority'], -x[0]['remaining_appetite']))
        return future[0]

    # Group supply by project
    from collections import defaultdict
    by_project = defaultdict(list)
    for r in supply_records:
        pname = r['Project Name']
        by_project[pname].append(r)

    projects = []
    for pname, recs in by_project.items():
        total_cost = sum(rr['Est Cash Amount'] for rr in recs)
        proj_date = min(r['Project_date'] for r in recs)
        proj_priority = min(r['Priority'] for r in recs)
        proj_gdf_profit = max(r['GDF Profit'] for r in recs)
        projects.append((pname, recs, total_cost, proj_date, proj_priority, proj_gdf_profit))

    # Sort projects by (Priority asc, GDF Profit desc, total_cost desc)
    projects.sort(key=lambda x: (x[4], -x[5], -x[2]))

    for (pname, recs, total_cost, pdate, proj_priority, proj_gdf_profit) in projects:
        temp_allocs = []
        temp_financed = []

        local_demand = [dict(d) for d in demand_records]
        local_bank = bank_available

        # Sort each project's chunks by (Priority asc, Est Cash desc)
        sorted_chunks = sorted(recs, key=lambda x: (x['Priority'], -x['Est Cash Amount']))

        project_ok = True

        for ch in sorted_chunks:
            chunk_cost = ch['Est Cash Amount']
            p_name = ch['Project Name']
            p_cat  = ch.get('Project Category','')
            p_st   = ch.get('Project State','')
            p_gdf  = ch['GDF Profit']
            p_id   = ch['Project ID']
            p_priority = ch['Priority']
            p_chunk_date = ch['Project_date']

            # 1. Try direct purchaser
            direct = find_direct_purchaser(p_chunk_date, chunk_cost, local_demand)
            if direct:
                pur, pur_idx = direct
                pur['remaining_appetite'] -= chunk_cost
                temp_allocs.append({
                    "Purchaser Name": pur['Purchaser Name'],
                    "Purchaser State": pur.get('Purchaser State',''),
                    "Purchaser Date": pur['Purchaser_Date'],
                    "Purchaser Category": pur.get('Purchaser Category',''),

                    "Project Name": p_name,
                    "Project Date": p_chunk_date,
                    "Project Category": p_cat,
                    "Project State": p_st,
                    "GDF Profit": p_gdf,
                    "Priority": p_priority,

                    "Allocated Amount": chunk_cost,
                    "Financed Amount": 0.0,
                    "Remaining Appetite After": pur['remaining_appetite'],
                    "Original Appetite": pur['approximated_appetite'],

                    "Project ID": p_id,
                    "Result": True
                })
                continue

            # 2. Bank bridging (only if bank_credit_limit>0)
            if local_bank >= chunk_cost and bank_credit_limit > 0:
                future = find_future_purchaser(p_chunk_date, chunk_cost, local_demand)
                if future:
                    f_pur, f_idx = future
                    local_bank -= chunk_cost

                    # Bank "entry"
                    temp_allocs.append({
                        "Purchaser Name": "Bank Credit",
                        "Purchaser State": "N/A",
                        "Purchaser Date": p_chunk_date,
                        "Purchaser Category": "N/A",

                        "Project Name": p_name,
                        "Project Date": p_chunk_date,
                        "Project Category": p_cat,
                        "Project State": p_st,
                        "GDF Profit": p_gdf,
                        "Priority": p_priority,

                        "Allocated Amount": chunk_cost,
                        "Financed Amount": chunk_cost,
                        "Remaining Appetite After": 0,
                        "Original Appetite": 0,

                        "Project ID": p_id,
                        "Result": True
                    })

                    # Bank "exit"
                    f_pur['remaining_appetite'] -= chunk_cost
                    local_bank += chunk_cost
                    exit_date = f_pur['Purchaser_Date']

                    temp_allocs.append({
                        "Purchaser Name": f_pur['Purchaser Name'],
                        "Purchaser State": f_pur.get('Purchaser State',''),
                        "Purchaser Date": exit_date,
                        "Purchaser Category": f_pur.get('Purchaser Category',''),

                        "Project Name": p_name,
                        "Project Date": p_chunk_date,
                        "Project Category": p_cat,
                        "Project State": p_st,
                        "GDF Profit": p_gdf,
                        "Priority": p_priority,

                        "Allocated Amount": chunk_cost,
                        "Financed Amount": -chunk_cost,
                        "Remaining Appetite After": f_pur['remaining_appetite'],
                        "Original Appetite": f_pur['approximated_appetite'],

                        "Project ID": p_id,
                        "Result": True
                    })

                    # Log bridging
                    temp_financed.append({
                        "Project ID": p_id,
                        "Project Name": p_name,
                        "Chunk Cost": chunk_cost,
                        "Purchase Date": p_chunk_date,
                        "Exit Date": exit_date,
                        "Exit Purchaser Name": f_pur['Purchaser Name'],
                        "Remaining Appetite After": f_pur['remaining_appetite'],
                        "Original Appetite": f_pur['approximated_appetite'],
                        "GDF Profit": p_gdf
                    })
                else:
                    project_ok = False
                    break
            else:
                # no direct purchaser + no bridging => fail this project
                project_ok = False
                break

        # If all chunks allocated => finalize
        if project_ok:
            allocated_projects.add(pname)
            demand_records = local_demand
            bank_available = local_bank
            allocations.extend(temp_allocs)
            financed_chunks.extend(temp_financed)
        else:
            # revert => skip entire project
            for ch in recs:
                allocations.append({
                    "Purchaser Name": None,
                    "Purchaser State": None,
                    "Purchaser Date": None,
                    "Purchaser Category": None,

                    "Project Name": ch['Project Name'],
                    "Project Date": ch['Project_date'],
                    "Project Category": ch.get('Project Category',''),
                    "Project State": ch.get('Project State',''),
                    "GDF Profit": ch['GDF Profit'],
                    "Priority": ch['Priority'],

                    "Allocated Amount": 0.0,
                    "Financed Amount": 0.0,
                    "Remaining Appetite After": None,
                    "Original Appetite": None,

                    "Project ID": ch['Project ID'],
                    "Result": False
                })

    # ---------------------------------------------------------------------
    # Build Allocations DF
    # ---------------------------------------------------------------------
    allocations_df = pd.DataFrame(allocations)
    alloc_cols = [
        "Purchaser Name","Purchaser State","Purchaser Date","Purchaser Category",
        "Project Name","Project Date","Project Category","Project State","GDF Profit",
        "Priority","Allocated Amount","Financed Amount","Remaining Appetite After",
        "Original Appetite","Project ID","Result"
    ]
    alloc_cols = [c for c in alloc_cols if c in allocations_df.columns]
    allocations_df = allocations_df[alloc_cols]

    # ---------------------------------------------------------------------
    # Build Financing DF
    # ---------------------------------------------------------------------
    financed_df = pd.DataFrame(financed_chunks)
    fin_cols = [
        "Project ID","Project Name","Chunk Cost","Purchase Date","Exit Date",
        "Exit Purchaser Name","Remaining Appetite After","Original Appetite",
        "GDF Profit"
    ]
    for cc in fin_cols:
        if cc not in financed_df.columns:
            financed_df[cc] = None
    financed_df = financed_df[fin_cols]

    # Now add the extra columns:
    #  Time of Bank Credit, Cost of Interest, Gross Profit, Net Profit, Net profit %
    if not financed_df.empty:
        financed_df['Purchase Date'] = pd.to_datetime(financed_df['Purchase Date'], errors='coerce')
        financed_df['Exit Date'] = pd.to_datetime(financed_df['Exit Date'], errors='coerce')

        # months between Purchase & Exit
        financed_df['Time of Bank Credit'] = (
            (financed_df['Exit Date'] - financed_df['Purchase Date']).dt.days // 30
        ).fillna(0).astype(int)

        # 1% monthly interest
        financed_df['Cost of Interest'] = financed_df['Chunk Cost'] * 0.01 * financed_df['Time of Bank Credit']
        financed_df['Gross Profit'] = financed_df['Chunk Cost'] * financed_df['GDF Profit']
        financed_df['Net Profit'] = financed_df['Gross Profit'] - financed_df['Cost of Interest']
        financed_df['Net profit %'] = financed_df['Net Profit'] / financed_df['Chunk Cost']
    else:
        financed_df['Time of Bank Credit'] = None
        financed_df['Cost of Interest'] = None
        financed_df['Gross Profit'] = None
        financed_df['Net Profit'] = None
        financed_df['Net profit %'] = None

    # ---------------------------------------------------------------------
    # Summary Metrics
    # ---------------------------------------------------------------------
    total_alloc = allocations_df['Allocated Amount'].sum()
    financed_used = allocations_df.loc[allocations_df['Financed Amount'] > 0, 'Financed Amount'].sum()
    bank_remaining = bank_available

    original_total_appetite = demand_df['approximated_appetite'].sum()
    demand_records_df = pd.DataFrame(demand_records)
    remaining_demand = demand_records_df['remaining_appetite'].sum()

    original_total_project_value = supply_df['Original Est Cash Amount'].sum()
    unallocated_projects = set(by_project.keys()) - allocated_projects
    unallocated_supply_df = supply_df[supply_df['Project Name'].isin(unallocated_projects)]
    remaining_supply_value = unallocated_supply_df['Est Cash Amount'].sum()

    pct_alloc = 0.0
    if original_total_project_value > 0:
        pct_alloc = (total_alloc / original_total_project_value) * 100

    summary_data = [
        {"Metric": "--- RESULTS SUMMARY ---", "Value": ""},
        {"Metric": "Total Allocated", "Value": f"{total_alloc:,.2f}"},
        {"Metric": "Financed (Bank Used)", "Value": f"{financed_used:,.2f}"},
        {"Metric": "Bank Credit Remaining", "Value": f"{bank_remaining:,.2f}"},

        {"Metric": "", "Value": ""},
        {"Metric": "DEMAND SUM", "Value": ""},
        {"Metric": "Original Total Appetite", "Value": f"{original_total_appetite:,.2f}"},
        {"Metric": "Remaining Demand", "Value": f"{remaining_demand:,.2f}"},

        {"Metric": "", "Value": ""},
        {"Metric": "% of Allocated Supply", "Value": f"{pct_alloc:.2f}%"},

        {"Metric": "", "Value": ""},
        {"Metric": "SUPPLY SUM", "Value": ""},
        {"Metric": "Original Total Project Value", "Value": f"{original_total_project_value:,.2f}"},
        {"Metric": "Remaining Supply", "Value": f"{remaining_supply_value:,.2f}"},

        {"Metric": "", "Value": ""},
        {"Metric": "--- GROSS PROFIT METRICS ---", "Value": ""},
        {"Metric": "Total Gross Profit (all projects)", "Value": ""},
        {"Metric": "Total Gross Profit Allocated", "Value": ""},
        {"Metric": "Total Gross Profit Unallocated", "Value": ""}
    ]
    summary_df = pd.DataFrame(summary_data)

    # ---------------------------------------------------------------------
    # Supply Summary with Gross Profit Metrics
    # ---------------------------------------------------------------------
    project_results = allocations_df.groupby('Project Name')['Result'].all().reset_index()
    project_results.rename(columns={'Result': 'Result'}, inplace=True)
    supply_after_df = supply_df.merge(project_results, on='Project Name', how='left')
    supply_after_df['Result'] = supply_after_df['Result'].fillna(False)

    supply_summary = supply_after_df.groupby('Project Name').agg(
        Total_Chunks_Before=('Est Cash Amount', 'count'),
        Total_Chunks_After=('Result', lambda x: x.sum()),
        Total_Project_Value=('Est Cash Amount', 'sum'),
        Latest_Project_Date=('Project_date', 'max'),
        Max_Est_Cash_Amount=('Est Cash Amount', 'max'),
        GDF_Profit=('GDF Profit', 'max'),
        Priority=('Priority', 'max')
    ).reset_index()

    supply_summary['Gross_Profit'] = (
        supply_summary['Total_Project_Value'] * supply_summary['GDF_Profit']
    )

    total_gross_profit = supply_summary['Gross_Profit'].sum()
    total_gross_profit_allocated = supply_summary.loc[supply_summary['Total_Chunks_After'] > 0, 'Gross_Profit'].sum()
    total_gross_profit_unallocated = supply_summary.loc[supply_summary['Total_Chunks_After'] == 0, 'Gross_Profit'].sum()

    # Populate summary with the final gross profit data
    summary_df.iloc[-3, summary_df.columns.get_loc("Value")] = f"{total_gross_profit:,.2f}"
    summary_df.iloc[-2, summary_df.columns.get_loc("Value")] = f"{total_gross_profit_allocated:,.2f}"
    summary_df.iloc[-1, summary_df.columns.get_loc("Value")] = f"{total_gross_profit_unallocated:,.2f}"

    # ---------------------------------------------------------------------
    # Build DemandAfter
    # ---------------------------------------------------------------------
    updated_demand_df = demand_records_df.copy()
    updated_demand_df['allocated_amount'] = (
        updated_demand_df['approximated_appetite'] - updated_demand_df['remaining_appetite']
    )
    updated_demand_df["Financing"] = 0.0

    if not allocations_df.empty and "Financed Amount" in allocations_df.columns:
        financing_by_purchaser = (
            allocations_df[allocations_df['Financed Amount'] < 0]
            .groupby('Purchaser Name')['Financed Amount'].sum()
            .abs()
            .reset_index(name='Financing')
        )
        updated_demand_df = updated_demand_df.merge(
            financing_by_purchaser, on='Purchaser Name', how='left', suffixes=('', '_calc')
        )
        updated_demand_df['Financing'] = updated_demand_df['Financing_calc'].fillna(0)
        updated_demand_df.drop(columns=['Financing_calc'], inplace=True)

    demand_after_cols = [
        "Purchaser Name",
        "approximated_appetite",
        "allocated_amount",
        "Purchaser State",
        "Purchaser_Date",
        "Financing",
        "remaining_appetite",
        "Purchaser Category",
        "State Priority"
    ]
    for c in demand_after_cols:
        if c not in updated_demand_df.columns:
            updated_demand_df[c] = None

    updated_demand_df = updated_demand_df[demand_after_cols]

    return (
        allocations_df,
        financed_df,
        summary_df,
        supply_summary,
        supply_after_df,
        updated_demand_df
    )

# -------------------------------------------------------------------------
# 9. Main Execution
# -------------------------------------------------------------------------
def main():
    # 1. Read data
    supply_df = fetch_supply()
    demand_df = fetch_demand()

    # 2. Allocate supply with bank_credit_limit=0 => no bridging
    (
        allocations_df,
        financed_df,
        summary_df,
        supply_summary,
        updated_supply,
        updated_demand_df
    ) = allocate_chunks_no_partial(supply_df, demand_df, bank_credit_limit=0_000_000)

    # 3. Print the summary DataFrame to the console
    print(summary_df.to_string(index=False))

    # 4. Save to Excel with multiple tabs
    with pd.ExcelWriter("allocation_results.xlsx", engine="xlsxwriter") as writer:
        allocations_df.to_excel(writer, sheet_name="Allocations", index=False)
        financed_df.to_excel(writer, sheet_name="Financing Chunks", index=False)
        summary_df.to_excel(writer, sheet_name="Summary", index=False)
        supply_summary.to_excel(writer, sheet_name="Supply Summary", index=False)
        updated_supply.to_excel(writer, sheet_name="SupplyAfter", index=False)
        updated_demand_df.to_excel(writer, sheet_name="DemandAfter", index=False)

    # 5. Notify user
    print("\nResults saved to 'allocation_results.xlsx' with:")
    print(" - Allocations (all direct + possible bank bridging if bank_credit_limit>0)")
    print(" - Financing Chunks (details of bridging, interest, etc.)")
    print(" - Summary (core metrics including total allocated & gross profit)")
    print(" - Supply Summary (project-level stats)")
    print(" - SupplyAfter (Fully Allocated flag)")
    print(" - DemandAfter (final appetite)")

if __name__ == "__main__":
    main()


                           Metric          Value
          --- RESULTS SUMMARY ---               
                  Total Allocated  34,988,752.47
             Financed (Bank Used)           0.00
            Bank Credit Remaining           0.00
                                                
                       DEMAND SUM               
          Original Total Appetite 120,414,628.00
                 Remaining Demand  85,425,875.53
                                                
            % of Allocated Supply         42.57%
                                                
                       SUPPLY SUM               
     Original Total Project Value  82,199,692.42
                 Remaining Supply  47,210,939.95
                                                
     --- GROSS PROFIT METRICS ---               
Total Gross Profit (all projects)  20,257,393.30
     Total Gross Profit Allocated   9,092,243.85
   Total Gross Profit Unallocated  11,165,149.45

Results saved to 'a

In [22]:
import gspread
from google.colab import auth
from google.auth import default
import pandas as pd
import numpy as np
from collections import defaultdict

# -------------------------------------------------------------------------
# 0. Authenticate and open the Google Sheet
# -------------------------------------------------------------------------
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

title = "2025 Project Agenda"
sheet = gc.open(title)

# -------------------------------------------------------------------------
# 1. Fetch Supply
# -------------------------------------------------------------------------
def fetch_supply():
    ws = sheet.worksheet("Supply")
    data = ws.get_all_values()
    supply_df = pd.DataFrame(data[1:], columns=data[0])

    # Keep only rows marked 'Y'
    supply_df = supply_df[supply_df['Take into consideration?'].str.lower() == 'y']

    # Convert numeric fields
    supply_df['Est Cash Amount'] = pd.to_numeric(
        supply_df['Est Cash Amount'].replace({',': ''}, regex=True)
    )
    supply_df['Project_date'] = pd.to_datetime(supply_df['Project_date'], errors='coerce')

    # Convert GDF Profit to numeric (e.g., "25%" -> 0.25)
    supply_df['GDF Profit'] = (
        supply_df['GDF Profit'].str.rstrip('%').astype(float) / 100
    )

    # Drop rows missing critical columns
    supply_df = supply_df.dropna(subset=['Est Cash Amount','Project_date','GDF Profit'])

    # Ensure we have a Project ID
    if 'Project ID' not in supply_df.columns or supply_df['Project ID'].isnull().all():
        supply_df['Project ID'] = supply_df['Project Name']
    else:
        supply_df['Project ID'] = supply_df['Project ID'].fillna(supply_df['Project Name'])

    # If needed, derive "Project Category" from chunk size
    if 'Project Category' not in supply_df.columns:
        bins = [0,350000,500000,800000,1000000,float('inf')]
        labels= ['0-350k','350k-500k','500k-800k','800k-1M','1M+']
        supply_df['Project Category'] = pd.cut(
            supply_df['Est Cash Amount'], bins=bins, labels=labels
        )

    # Rename "State" -> "Project State" if needed
    if 'State' in supply_df.columns:
        supply_df.rename(columns={'State':'Project State'}, inplace=True)

    # Ensure Priority is numeric and sort
    if 'Priority' not in supply_df.columns:
        raise ValueError("Supply data must contain a 'Priority' column.")

    supply_df['Priority'] = pd.to_numeric(supply_df['Priority'], errors='coerce')
    supply_df = supply_df.dropna(subset=['Priority'])

    # Sort supply in the same way as your non-financing code
    supply_df = supply_df.sort_values(
        by=['Priority', 'GDF Profit', 'Est Cash Amount', 'Project_date'],
        ascending=[True, False, False, True]
    )

    # Keep an original copy of Est Cash Amount
    supply_df['Original Est Cash Amount'] = supply_df['Est Cash Amount']

    # Add 'Fully Allocated' flag
    supply_df['Fully Allocated'] = False

    return supply_df

# -------------------------------------------------------------------------
# 2. Fetch Demand
# -------------------------------------------------------------------------
def fetch_demand():
    ws = sheet.worksheet("Demand")
    data = ws.get_all_values()
    demand_df = pd.DataFrame(data[1:], columns=data[0])

    # Rename columns if needed
    demand_df = demand_df.rename(columns={'purchaser_name': 'Purchaser Name'})

    # Convert numeric fields
    demand_df['approximated_appetite'] = pd.to_numeric(
        demand_df['approximated_appetite'].replace({',':''}, regex=True)
    )
    # Remove 'pending_amount' if it exists
    if 'pending_amount' in demand_df.columns:
        demand_df.drop(columns=['pending_amount'], inplace=True)

    # Convert dates
    demand_df['Purchaser_Date'] = pd.to_datetime(demand_df['Purchaser_Date'], errors='coerce')
    demand_df = demand_df.dropna(subset=['approximated_appetite','Purchaser_Date'])

    # Track remaining appetite
    demand_df['remaining_appetite'] = demand_df['approximated_appetite']

    # Rename "State" -> "Purchaser State" if needed
    if 'State' in demand_df.columns:
        demand_df.rename(columns={'State':'Purchaser State'}, inplace=True)

    # Ensure "Purchaser Category"
    if 'Purchaser Category' not in demand_df.columns:
        bins = [0,350000,500000,800000,1000000,float('inf')]
        labels= ['0-350k','350k-500k','500k-800k','800k-1M','1M+']
        demand_df['Purchaser Category'] = pd.cut(
            demand_df['approximated_appetite'], bins=bins, labels=labels
        )

    # State Priority: CA=1, others=2
    demand_df['State Priority'] = demand_df['Purchaser State'].apply(
        lambda x: 1 if x == 'CA' else 2
    )

    # Sort demand by appetite desc, then state priority
    demand_df = demand_df.sort_values(
        by=['approximated_appetite', 'State Priority'],
        ascending=[False, True]
    )

    return demand_df

# -------------------------------------------------------------------------
# 3. Allocate function (with optional bank bridging)
# -------------------------------------------------------------------------
def allocate_chunks_no_partial(
    supply_df: pd.DataFrame,
    demand_df: pd.DataFrame,
    bank_credit_limit: float = 0_000_000 # bank credit limit
):
    """
    Allocates supply to demand. If bank_credit_limit=0 => no bridging,
    matching the non-financing approach. If >0 => attempt bridging.
    CA purchasers (State Priority=1) are prioritized over others.
    """
    supply_df = supply_df.copy()
    demand_df = demand_df.copy()

    supply_records = supply_df.to_dict('records')
    demand_records = demand_df.to_dict('records')

    allocations = []
    financed_chunks = []
    bank_available = bank_credit_limit

    allocated_projects = set()

    # Updated: CA-first + largest appetite sorting for direct
    def find_direct_purchaser(project_date, chunk_cost, d_records):
        eligible = [
            (p, i) for i, p in enumerate(d_records)
            if (p['Purchaser_Date'] <= project_date) and (p['remaining_appetite'] >= chunk_cost)
        ]
        if not eligible:
            return None
        # Sort by (State Priority asc, remaining_appetite desc)
        eligible.sort(key=lambda x: (x[0]['State Priority'], -x[0]['remaining_appetite']))
        return eligible[0]

    # Updated: CA-first + largest appetite sorting for bridging
    def find_future_purchaser(project_date, chunk_cost, d_records):
        future = [
            (p, i) for i, p in enumerate(d_records)
            if (p['Purchaser_Date'] >= project_date) and (p['remaining_appetite'] >= chunk_cost)
        ]
        if not future:
            return None
        # Sort by (State Priority asc, remaining_appetite desc)
        future.sort(key=lambda x: (x[0]['State Priority'], -x[0]['remaining_appetite']))
        return future[0]

    # Group supply by Project Name in the same order they appear
    grouped_supply = supply_df.groupby('Project Name', sort=False)

    for project_name, group in grouped_supply:
        # If the entire project is already allocated, skip
        if group['Fully Allocated'].all():
            continue

        # Initialize project_ok for each project
        project_ok = True

        project_total_cost = group['Est Cash Amount'].sum()
        project_chunks = group.to_dict('records')  # convert group to list of chunk dicts

        temp_allocs = []
        temp_financed = []

        # Work on a copy of the global demand
        local_demand = [dict(d) for d in demand_records]
        local_bank = bank_available

        # Sort chunks internally to match non-financing logic:
        # By Priority asc, then Est Cash desc
        sorted_chunks = sorted(project_chunks,
                               key=lambda x: (x['Priority'], -x['Est Cash Amount']))

        # -------------------------------------
        # For each chunk in the current project
        # -------------------------------------
        for chunk in sorted_chunks:
            if chunk.get('Fully Allocated'):
                continue  # Already allocated

            chunk_cost = chunk['Est Cash Amount']
            p_name = chunk['Project Name']
            p_cat  = chunk.get('Project Category','')
            p_st   = chunk.get('Project State','')
            p_gdf  = chunk['GDF Profit']
            p_id   = chunk['Project ID']
            p_priority = chunk['Priority']
            p_chunk_date = chunk['Project_date']

            # 1) Try direct purchaser
            direct = find_direct_purchaser(p_chunk_date, chunk_cost, local_demand)
            if direct:
                pur, pur_idx = direct
                pur['remaining_appetite'] -= chunk_cost

                # Record the allocation
                temp_allocs.append({
                    "Purchaser Name": pur['Purchaser Name'],
                    "Purchaser State": pur.get('Purchaser State',''),
                    "Purchaser Date": pur['Purchaser_Date'],
                    "Purchaser Category": pur.get('Purchaser Category',''),

                    "Project Name": p_name,
                    "Project Date": p_chunk_date,
                    "Project Category": p_cat,
                    "Project State": p_st,
                    "GDF Profit": p_gdf,
                    "Priority": p_priority,

                    "Allocated Amount": chunk_cost,
                    "Financed Amount": 0.0,
                    "Remaining Appetite After": pur['remaining_appetite'],
                    "Original Appetite": pur['approximated_appetite'],

                    "Project ID": p_id,
                    "Result": True
                })
                continue

            # 2) If no direct buyer, attempt bridging if bank_credit_limit > 0
            if local_bank >= chunk_cost and bank_credit_limit > 0:
                future = find_future_purchaser(p_chunk_date, chunk_cost, local_demand)
                if future:
                    f_pur, f_idx = future
                    # Bank buys chunk now
                    local_bank -= chunk_cost

                    # Bank "entry"
                    temp_allocs.append({
                        "Purchaser Name": "Bank Credit",
                        "Purchaser State": "N/A",
                        "Purchaser Date": p_chunk_date,
                        "Purchaser Category": "N/A",

                        "Project Name": p_name,
                        "Project Date": p_chunk_date,
                        "Project Category": p_cat,
                        "Project State": p_st,
                        "GDF Profit": p_gdf,
                        "Priority": p_priority,

                        "Allocated Amount": chunk_cost,
                        "Financed Amount": chunk_cost,
                        "Remaining Appetite After": 0,
                        "Original Appetite": 0,

                        "Project ID": p_id,
                        "Result": True
                    })

                    # Bank "exit" once that future purchaser is available
                    f_pur['remaining_appetite'] -= chunk_cost
                    exit_date = f_pur['Purchaser_Date']
                    local_bank += chunk_cost

                    temp_allocs.append({
                        "Purchaser Name": f_pur['Purchaser Name'],
                        "Purchaser State": f_pur.get('Purchaser State',''),
                        "Purchaser Date": exit_date,
                        "Purchaser Category": f_pur.get('Purchaser Category',''),

                        "Project Name": p_name,
                        "Project Date": p_chunk_date,
                        "Project Category": p_cat,
                        "Project State": p_st,
                        "GDF Profit": p_gdf,
                        "Priority": p_priority,

                        "Allocated Amount": chunk_cost,
                        "Financed Amount": -chunk_cost,
                        "Remaining Appetite After": f_pur['remaining_appetite'],
                        "Original Appetite": f_pur['approximated_appetite'],

                        "Project ID": p_id,
                        "Result": True
                    })

                    # Track bridging info in financed_chunks
                    temp_financed.append({
                        "Project ID": p_id,
                        "Project Name": p_name,
                        "Chunk Cost": chunk_cost,
                        "Purchase Date": p_chunk_date,
                        "Exit Date": exit_date,
                        "Exit Purchaser Name": f_pur['Purchaser Name'],
                        "Remaining Appetite After": f_pur['remaining_appetite'],
                        "Original Appetite": f_pur['approximated_appetite'],
                        "GDF Profit": p_gdf
                    })
                else:
                    # Bridging not possible => fail project
                    project_ok = False
                    break
            else:
                # No direct buyer & bridging not possible => fail project
                project_ok = False
                break

        # If the project was fully allocated => finalize
        if project_ok:
            allocated_projects.add(project_name)
            # Accept local_demand changes
            demand_records = local_demand
            bank_available = local_bank

            allocations.extend(temp_allocs)
            financed_chunks.extend(temp_financed)

            # Mark supply "Fully Allocated"
            supply_df.loc[group.index, 'Fully Allocated'] = True
        else:
            # Revert => skip entire project
            for ch in group.to_dict('records'):
                allocations.append({
                    "Purchaser Name": None,
                    "Purchaser State": None,
                    "Purchaser Date": None,
                    "Purchaser Category": None,

                    "Project Name": ch['Project Name'],
                    "Project Date": ch['Project_date'],
                    "Project Category": ch.get('Project Category',''),
                    "Project State": ch.get('Project State',''),
                    "GDF Profit": ch['GDF Profit'],
                    "Priority": ch['Priority'],

                    "Allocated Amount": 0.0,
                    "Financed Amount": 0.0,
                    "Remaining Appetite After": None,
                    "Original Appetite": None,

                    "Project ID": ch['Project ID'],
                    "Result": False
                })
            # No need to delete 'project_ok' as it's re-initialized at the start of the loop

    # ---------------------------------------------------------------------
    # Build Allocations DF
    # ---------------------------------------------------------------------
    allocations_df = pd.DataFrame(allocations)
    alloc_cols = [
        "Purchaser Name","Purchaser State","Purchaser Date","Purchaser Category",
        "Project Name","Project Date","Project Category","Project State","GDF Profit",
        "Priority","Allocated Amount","Financed Amount","Remaining Appetite After",
        "Original Appetite","Project ID","Result"
    ]
    alloc_cols = [c for c in alloc_cols if c in allocations_df.columns]
    allocations_df = allocations_df[alloc_cols]

    # ---------------------------------------------------------------------
    # Build Financing DF
    # ---------------------------------------------------------------------
    financed_df = pd.DataFrame(financed_chunks)
    fin_cols = [
        "Project ID","Project Name","Chunk Cost","Purchase Date","Exit Date",
        "Exit Purchaser Name","Remaining Appetite After","Original Appetite",
        "GDF Profit"
    ]
    for cc in fin_cols:
        if cc not in financed_df.columns:
            financed_df[cc] = None
    financed_df = financed_df[fin_cols]

    # Add interest & profit calcs if bridging used
    if not financed_df.empty:
        financed_df['Purchase Date'] = pd.to_datetime(financed_df['Purchase Date'], errors='coerce')
        financed_df['Exit Date'] = pd.to_datetime(financed_df['Exit Date'], errors='coerce')

        # Calculate months between Purchase and Exit
        financed_df['Time of Bank Credit'] = (
            (financed_df['Exit Date'] - financed_df['Purchase Date']).dt.days // 30
        ).fillna(0).astype(int)

        # 1% monthly interest (as implied by your example)
        financed_df['Cost of Interest'] = (
            financed_df['Chunk Cost'] * 0.01 * financed_df['Time of Bank Credit']
        )

        financed_df['Gross Profit'] = (
            financed_df['Chunk Cost'] * financed_df['GDF Profit']
        )

        financed_df['Net Profit'] = (
            financed_df['Gross Profit'] - financed_df['Cost of Interest']
        )

        financed_df['Net profit %'] = financed_df['Net Profit'] / financed_df['Chunk Cost']
    else:
        # If no bridging occurred, just ensure columns exist
        financed_df['Time of Bank Credit'] = None
        financed_df['Cost of Interest'] = None
        financed_df['Gross Profit'] = None
        financed_df['Net Profit'] = None
        financed_df['Net profit %'] = None

    # ---------------------------------------------------------------------
    # Summary Metrics
    # ---------------------------------------------------------------------
    total_alloc = allocations_df['Allocated Amount'].sum()
    financed_used = allocations_df.loc[allocations_df['Financed Amount'] > 0, 'Financed Amount'].sum()
    bank_remaining = bank_available

    # Demand side
    original_total_appetite = demand_df['approximated_appetite'].sum()
    demand_records_df = pd.DataFrame(demand_records)
    remaining_demand = demand_records_df['remaining_appetite'].sum()

    # Supply side
    original_total_project_value = supply_df['Original Est Cash Amount'].sum()

    unallocated_projects = set(supply_df['Project Name'].unique()) - allocated_projects
    unallocated_supply_df = supply_df[supply_df['Project Name'].isin(unallocated_projects)]
    remaining_supply_value = unallocated_supply_df['Est Cash Amount'].sum()

    pct_alloc = 0.0
    if original_total_project_value > 0:
        pct_alloc = (total_alloc / original_total_project_value) * 100

    # Prepare "core" summary DataFrame
    summary_data = [
        {"Metric": "--- RESULTS SUMMARY ---", "Value": ""},
        {"Metric": "Total Allocated", "Value": f"{total_alloc:,.2f}"},
        {"Metric": "Financed (Bank Used)", "Value": f"{financed_used:,.2f}"},
        {"Metric": "Bank Credit Remaining", "Value": f"{bank_remaining:,.2f}"},

        {"Metric": "", "Value": ""},
        {"Metric": "DEMAND SUM", "Value": ""},
        {"Metric": "Original Total Appetite", "Value": f"{original_total_appetite:,.2f}"},
        {"Metric": "Remaining Demand", "Value": f"{remaining_demand:,.2f}"},

        {"Metric": "", "Value": ""},
        {"Metric": "% of Allocated Supply", "Value": f"{pct_alloc:.2f}%"},

        {"Metric": "", "Value": ""},
        {"Metric": "SUPPLY SUM", "Value": ""},
        {"Metric": "Original Total Project Value", "Value": f"{original_total_project_value:,.2f}"},
        {"Metric": "Remaining Supply", "Value": f"{remaining_supply_value:,.2f}"},

        {"Metric": "", "Value": ""},
        {"Metric": "--- GROSS PROFIT METRICS ---", "Value": ""},
        {"Metric": "Total Gross Profit (all projects)", "Value": ""},
        {"Metric": "Total Gross Profit Allocated", "Value": ""},
        {"Metric": "Total Gross Profit Unallocated", "Value": ""}
    ]
    summary_df = pd.DataFrame(summary_data)

    # ---------------------------------------------------------------------
    # Supply Summary with Gross Profit Metrics
    # ---------------------------------------------------------------------
    project_results = allocations_df.groupby('Project Name')['Result'].all().reset_index()
    project_results.rename(columns={'Result': 'Result'}, inplace=True)
    supply_after_df = supply_df.merge(project_results, on='Project Name', how='left')
    supply_after_df['Result'] = supply_after_df['Result'].fillna(False)

    supply_summary = supply_after_df.groupby('Project Name').agg(
        Total_Chunks_Before=('Est Cash Amount', 'count'),
        Total_Chunks_After=('Result', lambda x: x.sum()),
        Total_Project_Value=('Est Cash Amount', 'sum'),
        Latest_Project_Date=('Project_date', 'max'),
        Max_Est_Cash_Amount=('Est Cash Amount', 'max'),
        GDF_Profit=('GDF Profit', 'max'),
        Priority=('Priority', 'max')
    ).reset_index()

    supply_summary['Gross_Profit'] = (
        supply_summary['Total_Project_Value'] * supply_summary['GDF_Profit']
    )

    total_gross_profit = supply_summary['Gross_Profit'].sum()
    total_gross_profit_allocated = supply_summary.loc[
        supply_summary['Total_Chunks_After'] > 0, 'Gross_Profit'
    ].sum()
    total_gross_profit_unallocated = supply_summary.loc[
        supply_summary['Total_Chunks_After'] == 0, 'Gross_Profit'
    ].sum()

    # Populate summary with the final gross profit data
    summary_df.iloc[-3, summary_df.columns.get_loc("Value")] = f"{total_gross_profit:,.2f}"
    summary_df.iloc[-2, summary_df.columns.get_loc("Value")] = f"{total_gross_profit_allocated:,.2f}"
    summary_df.iloc[-1, summary_df.columns.get_loc("Value")] = f"{total_gross_profit_unallocated:,.2f}"

    # ---------------------------------------------------------------------
    # Build DemandAfter
    # ---------------------------------------------------------------------
    updated_demand_df = demand_records_df.copy()
    updated_demand_df['allocated_amount'] = (
        updated_demand_df['approximated_appetite'] - updated_demand_df['remaining_appetite']
    )
    updated_demand_df["Financing"] = 0.0

    # If bridging used, track how much each purchaser eventually financed
    if not allocations_df.empty and "Financed Amount" in allocations_df.columns:
        financing_by_purchaser = (
            allocations_df[allocations_df['Financed Amount'] < 0]
            .groupby('Purchaser Name')['Financed Amount'].sum()
            .abs()
            .reset_index(name='Financing')
        )
        updated_demand_df = updated_demand_df.merge(
            financing_by_purchaser, on='Purchaser Name', how='left', suffixes=('', '_calc')
        )
        updated_demand_df['Financing'] = updated_demand_df['Financing_calc'].fillna(0)
        updated_demand_df.drop(columns=['Financing_calc'], inplace=True)

    demand_after_cols = [
        "Purchaser Name",
        "approximated_appetite",
        "allocated_amount",
        "Purchaser State",
        "Purchaser_Date",
        "Financing",
        "remaining_appetite",
        "Purchaser Category",
        "State Priority"
    ]
    for c in demand_after_cols:
        if c not in updated_demand_df.columns:
            updated_demand_df[c] = None

    updated_demand_df = updated_demand_df[demand_after_cols]

    return (
        allocations_df,      # Detailed chunk-by-chunk allocations
        financed_df,         # Bank bridging info
        summary_df,          # High-level results summary
        supply_summary,      # Per-project supply summary
        supply_after_df,     # Supply with "Result" indicating whether allocated
        updated_demand_df    # Demand after final allocations/bridging
    )

# -------------------------------------------------------------------------
# 4. Main Execution
# -------------------------------------------------------------------------
def main():
    supply_df = fetch_supply()
    demand_df = fetch_demand()

    (
        allocations_df,
        financed_df,
        summary_df,
        supply_summary,
        updated_supply,
        updated_demand_df
    ) = allocate_chunks_no_partial(supply_df, demand_df, bank_credit_limit=0_000_000) # bank credit limit

    # Print summary to console
    print(summary_df.to_string(index=False))

    # Save to Excel with multiple tabs
    with pd.ExcelWriter("allocation_results.xlsx", engine="xlsxwriter") as writer:
        allocations_df.to_excel(writer, sheet_name="Allocations", index=False)
        financed_df.to_excel(writer, sheet_name="Financing Chunks", index=False)
        summary_df.to_excel(writer, sheet_name="Summary", index=False)
        supply_summary.to_excel(writer, sheet_name="Supply Summary", index=False)
        updated_supply.to_excel(writer, sheet_name="SupplyAfter", index=False)
        updated_demand_df.to_excel(writer, sheet_name="DemandAfter", index=False)

    print("\nResults saved to 'allocation_results.xlsx' with:")
    print(" - Allocations (all direct + bank bridging if applicable)")
    print(" - Financing Chunks (details of bridging, interest & profit columns)")
    print(" - Summary (result metrics including Gross Profit)")
    print(" - Supply Summary (detailed project allocations and Gross Profit)")
    print(" - SupplyAfter (with 'Result' column indicating allocation success)")
    print(" - DemandAfter (final states of demand)")

# Execute the main function
if __name__ == "__main__":
    main()


                           Metric          Value
          --- RESULTS SUMMARY ---               
                  Total Allocated  89,289,766.33
             Financed (Bank Used)  19,781,455.67
            Bank Credit Remaining  20,000,000.00
                                                
                       DEMAND SUM               
          Original Total Appetite 120,414,628.00
                 Remaining Demand  50,906,317.34
                                                
            % of Allocated Supply        108.63%
                                                
                       SUPPLY SUM               
     Original Total Project Value  82,199,692.42
                 Remaining Supply  12,691,381.76
                                                
     --- GROSS PROFIT METRICS ---               
Total Gross Profit (all projects)  20,257,393.30
     Total Gross Profit Allocated  17,738,462.77
   Total Gross Profit Unallocated   2,518,930.52

Results saved to 'a